In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/amex-v15/test_data_v15.parquet
/kaggle/input/amex-v15/train_data_v15.parquet
/kaggle/input/amexkkgp/test_data.parquet
/kaggle/input/amexkkgp/add_event.parquet
/kaggle/input/amexkkgp/data_dictionary.csv
/kaggle/input/amexkkgp/offer_metadata.parquet
/kaggle/input/amexkkgp/add_trans.parquet
/kaggle/input/amexkkgp/test_enriched.parquet
/kaggle/input/amexkkgp/train_enriched.parquet
/kaggle/input/amexkkgp/train_data.parquet


In [2]:
# import pandas as pd
# import numpy as np
# import dask.dataframe as dd
# import warnings

# warnings.filterwarnings('ignore')

# print("Loading datasets...")
# # --- 1. Load Datasets (Using Dask for Large Files) ---
# # Load smaller files with pandas
# stratified_train_df = pd.read_parquet('/kaggle/input/amexkkgp/train_data.parquet')
# test_df = pd.read_parquet('/kaggle/input/amexkkgp/test_data.parquet')
# offer_df = pd.read_parquet('/kaggle/input/amexkkgp/offer_metadata.parquet')

# # Load large event and transaction logs with Dask to prevent memory crashes
# try:
#     event_df_dask = dd.read_parquet('/kaggle/input/amexkkgp/add_event.parquet')
#     trans_df_dask = dd.read_parquet('/kaggle/input/amexkkgp/add_trans.parquet')
#     print("All datasets loaded successfully (using Dask for large files).")
# except Exception as e:
#     print(f"Error loading supplementary data: {e}")
#     # Terminate if supplementary data isn't available
#     exit()

In [3]:
# stratified_train_df.shape

In [4]:
# # --- 2. Feature Engineering from add_event_df (Offer-Level Only) ---
# print("\nStarting feature engineering on event data for OFFERS only...")

# # Create a 'clicked' column based on whether id7 is null
# event_df_dask['clicked'] = (~event_df_dask['id7'].isnull()).astype(int)

# # --- Offer-level event features ---
# # Group by offer (id3) and aggregate its historical performance
# offer_event_features_dask = event_df_dask.groupby('id3').agg(
#     offer_total_impressions=('id4', 'count'),
#     offer_total_clicks=('clicked', 'sum')
# )
# # Calculate offer's historical CTR
# offer_event_features_dask['offer_historical_ctr'] = (
#     offer_event_features_dask['offer_total_clicks'] / offer_event_features_dask['offer_total_impressions']
# ).fillna(0)

# # --- Execute Dask computation ---
# print("Computing aggregated offer features...")
# offer_event_features = offer_event_features_dask.compute().reset_index()
# print("Offer event features created.")

In [5]:
# # --- 3. NEW: Feature Engineering from add_trans_df (Industry-Level) ---
# print("\nStarting feature engineering on transaction data for INDUSTRIES...")
# trans_df_dask['f367'] = dd.to_numeric(trans_df_dask['f367'], errors='coerce')

# # Group by industry (id8) and aggregate transaction behavior
# industry_trans_features_dask = trans_df_dask.groupby('id8').agg(
#     industry_avg_spend=('f367', 'mean'),
#     industry_total_transactions=('f367', 'count'),
#     industry_unique_products=('f368', dd.Aggregation('nunique', chunk=lambda s: s.nunique(), agg=lambda s: s.nunique()))
# )
# print("Computing aggregated industry features...")
# industry_trans_features = industry_trans_features_dask.compute().reset_index()
# print("Industry transaction features created.")

In [6]:
# # --- 4. Feature Engineering from offer_metadata_df (Pandas) ---
# print("\nStarting feature engineering on offer metadata...")
# offer_df['id12'] = pd.to_datetime(offer_df['id12'], errors='coerce')
# offer_df['id13'] = pd.to_datetime(offer_df['id13'], errors='coerce')
# offer_df['offer_duration_days'] = (offer_df['id13'] - offer_df['id12']).dt.days

# offer_meta_features = offer_df[['id3', 'f375', 'f376', 'id10', 'id8', 'offer_duration_days']].rename(columns={
#     'f375': 'offer_redemption_freq',
#     'f376': 'offer_discount_rate',
#     'id10': 'offer_type_code' # Renamed to avoid confusion with industry
# })
# # Ensure the merge keys are the correct type
# offer_meta_features['id3'] = offer_meta_features['id3'].astype(str)
# offer_meta_features['id8'] = offer_meta_features['id8'].astype(str)
# industry_trans_features['id8'] = industry_trans_features['id8'].astype(str)

# # --- NEW: Merge industry features into offer metadata ---
# offer_meta_features = pd.merge(offer_meta_features, industry_trans_features, on='id8', how='left')
# print("Offer metadata enriched with industry transaction data.")

In [7]:
# # --- 5. Merge All New Features into Main DataFrames ---
# print("\nMerging all new features into training and test sets...")
# def enrich_dataframe(df):
#     """Merges all engineered features into a given dataframe."""
#     df['id3'] = df['id3'].astype(str)

#     # Now merging small, pre-computed pandas DataFrames
#     df = pd.merge(df, offer_event_features, on='id3', how='left')
#     df = pd.merge(df, offer_meta_features, on='id3', how='left')

#     return df

# train_enriched = enrich_dataframe(stratified_train_df)
# test_enriched = enrich_dataframe(test_df)

# print(f"Enriched training data shape: {train_enriched.shape}")
# print(f"Enriched test data shape: {test_enriched.shape}")


# # --- 6. Save the Enriched Datasets ---
# print("\nSaving enriched datasets to Parquet files...")
# try:
#     train_enriched.to_parquet('/kaggle/working/train_enriched_v10.parquet')
#     test_enriched.to_parquet('/kaggle/working/test_enriched_v10.parquet')
#     print("Successfully saved enriched data.")
# except Exception as e:
#     print(f"Error saving enriched data: {e}")

In [2]:
import pandas as pd
import numpy as np
import dask.dataframe as dd
import warnings
import gc
from sklearn.model_selection import train_test_split

warnings.filterwarnings('ignore')

print("--- STAGE 1: Loading Datasets ---")
# --- Load Datasets (Using Dask for Large Files) ---
# Define file paths
INPUT_DIR = '/kaggle/input/amexkkgp/'
WORKING_DIR = '/kaggle/working/'

# Load smaller files with pandas
full_train_df = pd.read_parquet(f'{INPUT_DIR}train_data.parquet')
# This is the competition's test set, which we will use for the final prediction
# prediction_df = pd.read_parquet(f'{INPUT_DIR}test_data.parquet') 
offer_df = pd.read_parquet(f'{INPUT_DIR}offer_metadata.parquet')

# Load large event and transaction logs with Dask to prevent memory crashes
try:
    event_df_dask = dd.read_parquet(f'{INPUT_DIR}add_event.parquet')
    trans_df_dask = dd.read_parquet(f'{INPUT_DIR}add_trans.parquet')
    print("All datasets loaded successfully (using Dask for large files).")
except Exception as e:
    print(f"Error loading supplementary data: {e}")
    # Terminate if supplementary data isn't available
    exit()

# =============================================================================
# STAGE 2: INTELLIGENT SAMPLING AND DATA SPLITTING
# =============================================================================
print("\n--- STAGE 2: Applying Intelligent Sampling (5:1 Ratio) and Splitting Data ---")

# Ensure the target column 'y' is numeric
full_train_df['y'] = pd.to_numeric(full_train_df['y'], errors='coerce')

# Separate positive (clicks) and negative (non-clicks) samples from the full training data
df_positive = full_train_df[full_train_df['y'] == 1].copy()
df_negative = full_train_df[full_train_df['y'] == 0].copy()

# Calculate the number of negative samples to keep (5 times the number of positives)
num_positive_samples = len(df_positive)
num_negative_samples_to_keep = num_positive_samples * 5

print(f"Original data: {num_positive_samples} positive samples, {len(df_negative)} negative samples.")
print(f"Sampling {num_negative_samples_to_keep} negative samples for a 5:1 ratio...")

# Randomly sample the negative instances
df_negative_sampled = df_negative.sample(n=num_negative_samples_to_keep, random_state=42)

# Combine the positive samples with the sampled negative samples to create our master sampled dataset
df_sampled = pd.concat([df_positive, df_negative_sampled])

print("\nSampling complete.")
print(f"Total size of sampled data before splitting: {df_sampled.shape}")

# --- Create a stratified train/test split from the sampled data ---
# This will be our new training set and a reliable, labeled hold-out test set
print("Creating a 75/25 stratified split for training and hold-out testing...")
y = df_sampled['y']
X = df_sampled.drop(columns=['y'])

# stratify=y ensures the 5:1 ratio is maintained in both the train and test sets
train_df, holdout_test_df = train_test_split(df_sampled, test_size=0.25, random_state=42, stratify=y)

print("\nSplitting complete.")
print(f"New training data shape: {train_df.shape}")
print(f"New hold-out test data shape: {holdout_test_df.shape}")
print("\nNew training data class distribution:")
print(train_df['y'].value_counts())
print("\nHold-out test data class distribution:")
print(holdout_test_df['y'].value_counts())


# =============================================================================
# STAGE 3: FOUNDATIONAL FEATURE ENGINEERING
# =============================================================================
print("\n--- STAGE 3: Starting Foundational Feature Engineering ---")

# --- Feature Engineering from add_event_df (Offer-Level) ---
print("Engineering offer-level features from event data...")
event_df_dask['clicked'] = (~event_df_dask['id7'].isnull()).astype(int)
offer_event_features_dask = event_df_dask.groupby('id3').agg(
    offer_total_impressions=('id4', 'count'),
    offer_total_clicks=('clicked', 'sum')
)
offer_event_features_dask['offer_historical_ctr'] = (
    offer_event_features_dask['offer_total_clicks'] / offer_event_features_dask['offer_total_impressions']
).fillna(0)
offer_event_features = offer_event_features_dask.compute().reset_index()
print("Offer event features created.")

# --- Feature Engineering from add_trans_df (Industry-Level) ---
print("Engineering industry-level features from transaction data...")
trans_df_dask['f367'] = dd.to_numeric(trans_df_dask['f367'], errors='coerce')
industry_trans_features_dask = trans_df_dask.groupby('id8').agg(
    industry_avg_spend=('f367', 'mean'),
    industry_total_transactions=('f367', 'count'),
    industry_unique_products=('f368', dd.Aggregation('nunique', chunk=lambda s: s.nunique(), agg=lambda s: s.nunique()))
)
industry_trans_features = industry_trans_features_dask.compute().reset_index()
print("Industry transaction features created.")

# --- Feature Engineering from offer_metadata_df ---
print("Engineering features from offer metadata...")
offer_df['id12'] = pd.to_datetime(offer_df['id12'], errors='coerce')
offer_df['id13'] = pd.to_datetime(offer_df['id13'], errors='coerce')
offer_df['offer_duration_days'] = (offer_df['id13'] - offer_df['id12']).dt.days

offer_meta_features = offer_df[['id3', 'f375', 'f376', 'id10', 'id8', 'offer_duration_days']].rename(columns={
    'f375': 'offer_redemption_freq',
    'f376': 'offer_discount_rate',
    'id10': 'offer_type_code'
})
offer_meta_features['id3'] = offer_meta_features['id3'].astype(str)
offer_meta_features['id8'] = offer_meta_features['id8'].astype(str)
industry_trans_features['id8'] = industry_trans_features['id8'].astype(str)
offer_meta_features = pd.merge(offer_meta_features, industry_trans_features, on='id8', how='left')
print("Offer metadata enriched with industry data.")

# --- Clean up Dask DataFrames ---
del event_df_dask, trans_df_dask
gc.collect()

# =============================================================================
# STAGE 4: MERGE FEATURES AND SAVE
# =============================================================================
print("\n--- STAGE 4: Merging all new features and saving data ---")
def enrich_dataframe(df):
    """Merges all engineered features into a given dataframe."""
    df['id3'] = df['id3'].astype(str)
    df = pd.merge(df, offer_event_features, on='id3', how='left')
    df = pd.merge(df, offer_meta_features, on='id3', how='left')
    return df

# Enrich all three dataframes
train_enriched = enrich_dataframe(train_df)
test_enriched = enrich_dataframe(holdout_test_df)
# prediction_enriched = enrich_dataframe(prediction_df)


print(f"Enriched training data shape: {train_enriched.shape}")
print(f"Enriched holdout-test data shape: {test_enriched.shape}")
# print(f"Enriched prediction data shape: {prediction_enriched.shape}")

# --- Save the Enriched Datasets ---
print("Saving final datasets to Parquet files...")
try:
    train_enriched.to_parquet(f'{WORKING_DIR}train_final.parquet')
    test_enriched.to_parquet(f'{WORKING_DIR}test_final.parquet')
    # prediction_enriched.to_parquet(f'{WORKING_DIR}prediction_final.parquet')
    print("Successfully saved all three final datasets.")
except Exception as e:
    print(f"Error saving data: {e}")



--- STAGE 1: Loading Datasets ---
All datasets loaded successfully (using Dask for large files).

--- STAGE 2: Applying Intelligent Sampling (5:1 Ratio) and Splitting Data ---
Original data: 37051 positive samples, 733113 negative samples.
Sampling 185255 negative samples for a 5:1 ratio...

Sampling complete.
Total size of sampled data before splitting: (222306, 372)
Creating a 75/25 stratified split for training and hold-out testing...

Splitting complete.
New training data shape: (166729, 372)
New hold-out test data shape: (55577, 372)

New training data class distribution:
y
0    138941
1     27788
Name: count, dtype: int64

Hold-out test data class distribution:
y
0    46314
1     9263
Name: count, dtype: int64

--- STAGE 3: Starting Foundational Feature Engineering ---
Engineering offer-level features from event data...
Offer event features created.
Engineering industry-level features from transaction data...
Industry transaction features created.
Engineering features from offer 

In [3]:
train_enriched.shape

(166729, 383)

In [4]:
train_enriched.head()

,id1,id2,id3,id4,id5,y,f1,f2,f3,f4,...,offer_total_clicks,offer_historical_ctr,offer_redemption_freq,offer_discount_rate,offer_type_code,id8,offer_duration_days,industry_avg_spend,industry_total_transactions,industry_unique_products
0,1309455_654480_16-23_2023-11-01 05:29:18.734,1309455,654480,2023-11-01 05:29:18.734,2023-11-01,0,None,None,None,None,...,1098,0.016595,1.0,NaN,1,59991300,165.0,167.946222,14005.0,1.0
1,1277975_66290365_16-23_2023-11-01 10:00:06.073,1277975,66290365,2023-11-01 10:00:06.073,2023-11-01,0,5.0,None,13.0,88.0,...,719,0.054272,2.0,2.0,1,56610000,29.0,373.318764,11470.0,1.0
2,1756831_783808_16-23_2023-11-02 07:45:41.794,1756831,783808,2023-11-02 07:45:41.794,2023-11-02,0,None,None,None,None,...,358,0.008301,1.0,NaN,1,54990000,108.0,81.011179,61048.0,1.0
3,1839978_80893_16-23_2023-11-02 07:03:16.575,1839978,80893,2023-11-02 07:03:16.575,2023-11-02,0,48.0,25.0,22.0,29.0,...,852,0.058545,1.0,NaN,1,59630000,90.0,316.704859,11749.0,1.0
4,1382370_331980_16-23_2023-11-03 07:18:32.167,1382370,331980,2023-11-03 07:18:32.167,2023-11-03,0,None,40.0,None,None,...,4360,0.071267,1.0,NaN,1,56990300,69.0,178.032686,13629.0,1.0


In [5]:
import pandas as pd
import numpy as np
import warnings
import gc
import os

# Modeling libraries
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans

# Suppress warnings for cleaner output
warnings.filterwarnings('ignore')

# --- Define Kaggle File Paths ---
INPUT_DIR = '/kaggle/input/amexkkgp/'
WORKING_DIR = '/kaggle/working/'

# =============================================================================
# STAGE 1: LOAD DATA AND DATA DICTIONARY
# =============================================================================
print("--- STAGE 1: Loading Data ---")
try:
    # Load the datasets created by the previous sampling script
    df_train = pd.read_parquet(f'{WORKING_DIR}train_final.parquet')
    df_holdout_test = pd.read_parquet(f'{WORKING_DIR}test_final.parquet')
    
    # Load the data dictionary
    data_dict = pd.read_csv(f'{INPUT_DIR}data_dictionary.csv')
    
    print("Successfully loaded all required data.")
    print(f"Training data shape: {df_train.shape}")
    print(f"Hold-out test data shape: {df_holdout_test.shape}")

except FileNotFoundError:
    print("Error: Required data not found. Please run the sampling_and_feature_engineering.py script first.")
    exit()

# =============================================================================
# STAGE 2: DEFINE THE REUSABLE CLUSTERING FUNCTION
# =============================================================================
print("\n--- STAGE 2: Defining the Clustering Pipeline Function ---")

def create_cluster_features(df_train, df_holdout, data_dict, keywords, cluster_col_name, n_clusters=5):
    """
    Performs a full PCA + K-Means clustering pipeline.
    It FITS on the training data and TRANSFORMS both train and holdout datasets.
    """
    print(f"\n--- Creating cluster feature: '{cluster_col_name}' ---")
    
    # --- a. Select Features ---
    print(f"Selecting features based on keywords: {keywords}...")
    try:
        if keywords == 'all':
            group_features_in_df = [col for col in df_train.columns if col.startswith('f')]
        else:
            feature_name_col = 'masked_column'
            description_col = 'Description'
            group_mask = data_dict[description_col].str.contains('|'.join(keywords), case=False, na=False)
            group_features = data_dict[group_mask][feature_name_col].tolist()
            group_features_in_df = [col for col in group_features if col in df_train.columns]
        
        if not group_features_in_df:
            print(f"Warning: No features found for group '{cluster_col_name}'. Skipping.")
            return df_train, df_holdout
            
        print(f"Found {len(group_features_in_df)} features for this group.")
        # Ensure consistent column order
        X_train_cluster = df_train[group_features_in_df].copy()
        X_holdout_cluster = df_holdout[group_features_in_df].copy()

    except Exception as e:
        print(f"Error selecting features: {e}. Skipping this cluster.")
        return df_train, df_holdout

    # --- b. Preprocess Data ---
    print("Preprocessing data (impute, scale)...")
    for col in X_train_cluster.columns:
        X_train_cluster[col] = pd.to_numeric(X_train_cluster[col], errors='coerce')
        X_holdout_cluster[col] = pd.to_numeric(X_holdout_cluster[col], errors='coerce')

    imputer = SimpleImputer(strategy='median')
    scaler = StandardScaler()
    
    # Fit ONLY on training data, then transform both
    X_train_imputed = imputer.fit_transform(X_train_cluster)
    X_holdout_imputed = imputer.transform(X_holdout_cluster)

    X_train_scaled = scaler.fit_transform(X_train_imputed)
    X_holdout_scaled = scaler.transform(X_holdout_imputed)

    # --- c. PCA ---
    print("Applying PCA...")
    pca = PCA(n_components=0.90) # Keep components explaining 90% of variance
    pca.fit(X_train_scaled)
    
    X_train_pca = pca.transform(X_train_scaled)
    X_holdout_pca = pca.transform(X_holdout_scaled)
    print(f"PCA resulted in {pca.n_components_} components.")

    # --- d. K-Means Clustering ---
    print("Applying K-Means...")
    kmeans = KMeans(n_clusters=n_clusters, random_state=42, n_init='auto')
    
    # Fit ONLY on training data, then predict for both
    df_train[cluster_col_name] = kmeans.fit_predict(X_train_pca)
    df_holdout[cluster_col_name] = kmeans.predict(X_holdout_pca)
    
    print(f"Successfully created and added '{cluster_col_name}' feature to both datasets.")
    
    return df_train, df_holdout


# =============================================================================
# STAGE 3: DEFINE FEATURE GROUPS AND RUN THE PIPELINE
# =============================================================================
print("\n--- STAGE 3: Running the Clustering Pipeline for Each Feature Group ---")

# --- Define Keyword Groups ---
# Group 1: Customer Spending Behavior
spending_keywords = ['debit amount', 'debit transaction', 'spend in']
df_train, df_holdout_test = create_cluster_features(df_train, df_holdout_test, data_dict, spending_keywords, 'spending_cluster')

# Group 2: Customer Click/Impression History
history_keywords = ['ctr in last', 'clicks in last', 'impressions in last']
df_train, df_holdout_test = create_cluster_features(df_train, df_holdout_test, data_dict, history_keywords, 'history_cluster')

# Group 3: Customer Profile & Loyalty
profile_keywords = ['membership level', 'account age', 'interest score', 'miles', 'segments']
df_train, df_holdout_test = create_cluster_features(df_train, df_holdout_test, data_dict, profile_keywords, 'profile_cluster')

# Group 4: Customer Engagement on Amex Platforms
engagement_keywords = ['time spent', 'pages viewed', 'visited', 'logon']
df_train, df_holdout_test = create_cluster_features(df_train, df_holdout_test, data_dict, engagement_keywords, 'engagement_cluster')

# Group 5: All 'f' Features (Holistic)
df_train, df_holdout_test = create_cluster_features(df_train, df_holdout_test, data_dict, 'all', 'holistic_cluster')


# =============================================================================
# STAGE 4: SAVE THE FINAL ENRICHED DATASETS
# =============================================================================
print("\n--- STAGE 4: Saving Final Datasets with New Cluster Features ---")

# --- Verify the new columns were added ---
new_cols = ['spending_cluster', 'history_cluster', 'profile_cluster', 'engagement_cluster', 'holistic_cluster']
print("\nColumns added to the training data:")
print([col for col in new_cols if col in df_train.columns])
print("\nColumns added to the holdout-test data:")
print([col for col in new_cols if col in df_holdout_test.columns])

# --- Save the final DataFrames ---
try:
    train_output_path = f'{WORKING_DIR}train_clustered.parquet'
    holdout_test_output_path = f'{WORKING_DIR}holdout_test_clustered.parquet'
    
    df_train.to_parquet(train_output_path)
    df_holdout_test.to_parquet(holdout_test_output_path)
    
    print(f"\nSuccessfully saved final training data to: {train_output_path}")
    print(f"Successfully saved final holdout-test data to: {holdout_test_output_path}")
except Exception as e:
    print(f"\nError saving the final files: {e}")

print("\n--- CUSTOMER CLUSTER FEATURE ENGINEERING COMPLETE ---")

--- STAGE 1: Loading Data ---
Successfully loaded all required data.
Training data shape: (166729, 383)
Hold-out test data shape: (55577, 383)

--- STAGE 2: Defining the Clustering Pipeline Function ---

--- STAGE 3: Running the Clustering Pipeline for Each Feature Group ---

--- Creating cluster feature: 'spending_cluster' ---
Selecting features based on keywords: ['debit amount', 'debit transaction', 'spend in']...
Found 47 features for this group.
Preprocessing data (impute, scale)...
Applying PCA...
PCA resulted in 31 components.
Applying K-Means...
Successfully created and added 'spending_cluster' feature to both datasets.

--- Creating cluster feature: 'history_cluster' ---
Selecting features based on keywords: ['ctr in last', 'clicks in last', 'impressions in last']...
Found 49 features for this group.
Preprocessing data (impute, scale)...
Applying PCA...
PCA resulted in 28 components.
Applying K-Means...
Successfully created and added 'history_cluster' feature to both datasets.

In [6]:
import pandas as pd
import numpy as np
import warnings
import gc
import os

# Suppress warnings for cleaner output
warnings.filterwarnings('ignore')

# --- Define Kaggle File Paths ---
INPUT_DIR = '/kaggle/input/amexkkgp/'
WORKING_DIR = '/kaggle/working/'

# =============================================================================
# STAGE 1: LOAD LATEST DATASETS
# =============================================================================
print("--- STAGE 1: Loading Latest Datasets ---")
try:
    # Load your latest training and holdout test sets with cluster features
    df_train = pd.read_parquet(f'{WORKING_DIR}train_clustered.parquet')
    df_holdout_test = pd.read_parquet(f'{WORKING_DIR}holdout_test_clustered.parquet')
    
    print("Successfully loaded all required data.")
    print(f"Input training data shape: {df_train.shape}")
    print(f"Input hold-out test data shape: {df_holdout_test.shape}")

except FileNotFoundError:
    print("Error: Required clustered data not found. Please run the customer_clustering.py script first.")
    exit()

# =============================================================================
# STAGE 2: CREATE TIME-BASED AND INTERACTION FEATURES
# =============================================================================
print("\n--- STAGE 2: Creating Time-Based and Interaction Features ---")

def create_advanced_features(df):
    """A reusable function to create time-based and interaction features."""
    
    # --- Time-Based Features ---
    df['impression_time'] = pd.to_datetime(df['id4'], errors='coerce')
    df['hour_of_day'] = df['impression_time'].dt.hour
    df['day_of_week'] = df['impression_time'].dt.dayofweek # Monday=0, Sunday=6
    df['is_weekend'] = (df['day_of_week'] >= 5).astype(int)
    
    # --- Cyclical Time Features ---
    # This helps the model understand the cyclical nature of time (e.g., hour 23 is close to hour 0)
    df['hour_sin'] = np.sin(2 * np.pi * df['hour_of_day']/24.0)
    df['hour_cos'] = np.cos(2 * np.pi * df['hour_of_day']/24.0)
    df['day_sin'] = np.sin(2 * np.pi * df['day_of_week']/7.0)
    df['day_cos'] = np.cos(2 * np.pi * df['day_of_week']/7.0)

    # --- Interaction Features ---
    # We create these based on hypotheses from our EDA.
    # For example, is a certain offer type more effective on the weekend?
    if 'offer_type_code' in df.columns:
        # Convert to string to handle potential mixed types and ensure concatenation works
        df['offer_type_x_weekend'] = df['offer_type_code'].astype(str) + "_" + df['is_weekend'].astype(str)
    
    # How does an offer's historical popularity compare to the average for its type?
    if 'offer_historical_ctr' in df.columns and 'offer_type_code' in df.columns:
        # Use transform to broadcast the mean back to the original dataframe shape
        avg_ctr_by_type = df.groupby('offer_type_code')['offer_historical_ctr'].transform('mean')
        df['offer_ctr_vs_type_avg'] = df['offer_historical_ctr'] / (avg_ctr_by_type + 1e-6) # Add epsilon to avoid division by zero

    # This column is no longer needed after feature extraction
    df = df.drop(columns=['impression_time'])
    
    return df

# --- Add features to both the training and holdout test dataframes ---
print("Enriching training data with new features...")
df_train_final = create_advanced_features(df_train)
print("Enriching hold-out test data with new features...")
df_holdout_test_final = create_advanced_features(df_holdout_test)


# =============================================================================
# STAGE 3: SAVE THE FINAL ENRICHED DATASETS
# =============================================================================
print("\n--- STAGE 3: Saving Final Datasets with All New Features ---")

# --- Verify the new columns were added ---
new_cols = [
    'hour_of_day', 'day_of_week', 'is_weekend',
    'hour_sin', 'hour_cos', 'day_sin', 'day_cos',
    'offer_type_x_weekend', 'offer_ctr_vs_type_avg'
]
print("\nColumns added to the training data:")
print([col for col in new_cols if col in df_train_final.columns])
print("\nColumns added to the holdout-test data:")
print([col for col in new_cols if col in df_holdout_test_final.columns])

# --- Save the final DataFrames ---
try:
    train_output_path = f'{WORKING_DIR}train_time_features.parquet'
    holdout_test_output_path = f'{WORKING_DIR}holdout_test_time_features.parquet'
    
    df_train_final.to_parquet(train_output_path)
    df_holdout_test_final.to_parquet(holdout_test_output_path)
    
    print(f"\nSuccessfully saved final training data to: {train_output_path}")
    print(f"Successfully saved final holdout-test data to: {holdout_test_output_path}")
except Exception as e:
    print(f"\nError saving the final files: {e}")

print("\n--- TIME-BASED FEATURE ENGINEERING COMPLETE ---")

--- STAGE 1: Loading Latest Datasets ---
Successfully loaded all required data.
Input training data shape: (166729, 388)
Input hold-out test data shape: (55577, 388)

--- STAGE 2: Creating Time-Based and Interaction Features ---
Enriching training data with new features...
Enriching hold-out test data with new features...

--- STAGE 3: Saving Final Datasets with All New Features ---

Columns added to the training data:
['hour_of_day', 'day_of_week', 'is_weekend', 'hour_sin', 'hour_cos', 'day_sin', 'day_cos', 'offer_type_x_weekend', 'offer_ctr_vs_type_avg']

Columns added to the holdout-test data:
['hour_of_day', 'day_of_week', 'is_weekend', 'hour_sin', 'hour_cos', 'day_sin', 'day_cos', 'offer_type_x_weekend', 'offer_ctr_vs_type_avg']

Successfully saved final training data to: /kaggle/working/train_time_features.parquet
Successfully saved final holdout-test data to: /kaggle/working/holdout_test_time_features.parquet

--- TIME-BASED FEATURE ENGINEERING COMPLETE ---


In [7]:
df_train_final.shape

(166729, 397)

In [8]:
import pandas as pd
import numpy as np
import dask.dataframe as dd
import warnings
import gc
import os

# Suppress warnings for cleaner output
warnings.filterwarnings('ignore')

# --- Define Kaggle File Paths ---
INPUT_DIR = '/kaggle/input/amexkkgp/'
WORKING_DIR = '/kaggle/working/'

# =============================================================================
# STAGE 1: LOAD ALL NECESSARY DATA
# =============================================================================
print("--- STAGE 1: Loading All Necessary Data ---")
try:
    # Load the datasets with cluster features
    df_train = pd.read_parquet(f'{WORKING_DIR}train_clustered.parquet')
    df_holdout_test = pd.read_parquet(f'{WORKING_DIR}holdout_test_clustered.parquet')
    
    # Load supplementary data for feature creation
    offer_meta_df = pd.read_parquet(f'{INPUT_DIR}offer_metadata.parquet')
    event_dd = dd.read_parquet(f'{INPUT_DIR}add_event.parquet')
    trans_dd = dd.read_parquet(f'{INPUT_DIR}add_trans.parquet')
    
    print("Successfully loaded all required data.")
    print(f"Input training data shape: {df_train.shape}")
    print(f"Input holdout-test data shape: {df_holdout_test.shape}")

except FileNotFoundError:
    print("Error: Required data not found. Please ensure the customer_clustering.py script ran successfully.")
    exit()

# =============================================================================
# STAGE 2: CREATE NEW ADVANCED FEATURES
# =============================================================================
print("\n--- STAGE 2: Creating New Advanced Features ---")

# --- Feature 1: Average Time-to-Click per Offer ---
print("Creating 'Average Time-to-Click' feature...")
event_dd['impression_time'] = dd.to_datetime(event_dd['id4'], errors='coerce')
event_dd['click_time'] = dd.to_datetime(event_dd['id7'], errors='coerce')
event_dd['time_to_click_seconds'] = (event_dd['click_time'] - event_dd['impression_time']).dt.total_seconds()
avg_time_to_click = event_dd.groupby('id3')['time_to_click_seconds'].mean().compute().reset_index()
avg_time_to_click = avg_time_to_click.rename(columns={'time_to_click_seconds': 'avg_offer_time_to_click_seconds'})
print("'Average Time-to-Click' feature created.")

# --- Feature 2 & 3: Debit/Credit and Transaction Time Patterns per Industry ---
print("Creating industry-level transaction pattern features...")
trans_dd['f371_hour'] = dd.to_datetime(trans_dd['f371'], format='%H:%M:%S', errors='coerce').dt.hour
debit_credit_counts = trans_dd.groupby(['id8', 'f369']).size().compute().unstack(fill_value=0)
if 'C' not in debit_credit_counts.columns: debit_credit_counts['C'] = 0
if 'D' not in debit_credit_counts.columns: debit_credit_counts['D'] = 0
debit_credit_counts.columns = ['credit_trans_count', 'debit_trans_count']
avg_trans_hour = trans_dd.groupby('id8')['f371_hour'].mean().compute().reset_index(name='avg_trans_hour')

industry_pattern_features = pd.merge(debit_credit_counts.reset_index(), avg_trans_hour, on='id8', how='outer')
print("Industry pattern features created.")

# --- Feature 4: Offer Body Length ---
print("Creating 'Offer Body Length' feature...")
offer_meta_df['offer_body_length'] = offer_meta_df['f378'].str.len()
offer_body_feature = offer_meta_df[['id3', 'offer_body_length']]
print("'Offer Body Length' feature created.")

# Clean up memory
del event_dd, trans_dd
gc.collect()

# =============================================================================
# STAGE 3: MERGE ALL NEW FEATURES INTO MAIN DATASETS
# =============================================================================
print("\n--- STAGE 3: Merging All New Features ---")

def enrich_dataframe(df, avg_time_to_click, industry_pattern_features, offer_body_feature):
    """A reusable function to enrich a dataframe with all new features."""
    
    # Ensure merge keys are the correct type
    df['id3'] = df['id3'].astype(str)
    df['id8'] = df['id8'].astype(str)
    avg_time_to_click['id3'] = avg_time_to_click['id3'].astype(str)
    industry_pattern_features['id8'] = industry_pattern_features['id8'].astype(str)
    offer_body_feature['id3'] = offer_body_feature['id3'].astype(str)

    # Merge Time-to-Click
    df = pd.merge(df, avg_time_to_click, on='id3', how='left')
    
    # Merge Industry Patterns directly using the 'id8' column already in the dataframe
    df = pd.merge(df, industry_pattern_features, on='id8', how='left')

    # Merge Offer Body Length
    df = pd.merge(df, offer_body_feature, on='id3', how='left')
    
    return df

# --- Enrich both the training and holdout test dataframes ---
print("Enriching training data...")
df_train_final = enrich_dataframe(df_train, avg_time_to_click, industry_pattern_features, offer_body_feature)
print("Enriching hold-out test data...")
df_holdout_test_final = enrich_dataframe(df_holdout_test, avg_time_to_click, industry_pattern_features, offer_body_feature)

# =============================================================================
# STAGE 4: SAVE THE FINAL ENRICHED DATASETS
# =============================================================================
print("\n--- STAGE 4: Saving Final Datasets ---")

# --- Verify the new columns were added ---
new_cols = [
    'avg_offer_time_to_click_seconds', 'credit_trans_count', 'debit_trans_count',
    'avg_trans_hour', 'offer_body_length'
]
print("\nColumns added to the training data:")
print([col for col in new_cols if col in df_train_final.columns])
print("\nColumns added to the holdout-test data:")
print([col for col in new_cols if col in df_holdout_test_final.columns])

# --- Save the final DataFrames ---
try:
    train_output_path = f'{WORKING_DIR}train_advanced_features.parquet'
    holdout_test_output_path = f'{WORKING_DIR}holdout_test_advanced_features.parquet'
    
    df_train_final.to_parquet(train_output_path)
    df_holdout_test_final.to_parquet(holdout_test_output_path)
    
    print(f"\nSuccessfully saved final training data to: {train_output_path}")
    print(f"Successfully saved final holdout-test data to: {holdout_test_output_path}")
except Exception as e:
    print(f"\nError saving the final files: {e}")

print("\n--- ADVANCED HISTORICAL FEATURE ENGINEERING COMPLETE ---")


--- STAGE 1: Loading All Necessary Data ---
Successfully loaded all required data.
Input training data shape: (166729, 388)
Input holdout-test data shape: (55577, 388)

--- STAGE 2: Creating New Advanced Features ---
Creating 'Average Time-to-Click' feature...
'Average Time-to-Click' feature created.
Creating industry-level transaction pattern features...
Industry pattern features created.
Creating 'Offer Body Length' feature...
'Offer Body Length' feature created.

--- STAGE 3: Merging All New Features ---
Enriching training data...
Enriching hold-out test data...

--- STAGE 4: Saving Final Datasets ---

Columns added to the training data:
['avg_offer_time_to_click_seconds', 'credit_trans_count', 'debit_trans_count', 'avg_trans_hour', 'offer_body_length']

Columns added to the holdout-test data:
['avg_offer_time_to_click_seconds', 'credit_trans_count', 'debit_trans_count', 'avg_trans_hour', 'offer_body_length']

Successfully saved final training data to: /kaggle/working/train_advanced

In [9]:
import pandas as pd
import numpy as np
import warnings
import gc
import os

# Suppress warnings for cleaner output
warnings.filterwarnings('ignore')

# --- Define Kaggle File Paths ---
INPUT_DIR = '/kaggle/input/amexkkgp/'
WORKING_DIR = '/kaggle/working/'

# =============================================================================
# STAGE 1: LOAD THE LATEST TRAINING DATA AND IDENTIFY ZERO-VARIANCE COLUMNS
# =============================================================================
print("--- STAGE 1: Analyzing Training Data for Zero-Variance Features ---")
try:
    # Load your latest training set with advanced features
    df_train = pd.read_parquet(f'{WORKING_DIR}train_advanced_features.parquet')
    print("Successfully loaded train_advanced_features.parquet.")
    print(f"Original training data shape: {df_train.shape}")
except FileNotFoundError:
    print("Error: train_advanced_features.parquet not found. Please ensure the previous feature engineering script ran successfully.")
    exit()

# --- Identify columns with zero variance based on the training data ---
# A column is considered zero-variance if it has only one unique value and no NaNs.
cols_to_drop = []
for col in df_train.columns:
    # Skip identifier columns and the target variable
    if col in ['id1', 'id2', 'id3', 'id4', 'id5', 'y']:
        continue
    
    # Condition: exactly one unique value AND no missing values.
    if df_train[col].nunique() == 1 and df_train[col].isnull().sum() == 0:
        cols_to_drop.append(col)

print(f"\nFound {len(cols_to_drop)} columns with zero variance to remove.")
if cols_to_drop:
    print("Columns to be removed:", cols_to_drop)

# --- Drop the identified columns from the training set ---
df_train_cleaned = df_train.drop(columns=cols_to_drop)
print(f"\nNew training data shape after removing zero-variance columns: {df_train_cleaned.shape}")


# =============================================================================
# STAGE 2: APPLY THE SAME CLEANING TO THE HOLD-OUT TEST DATA
# =============================================================================
print("\n--- STAGE 2: Applying the Same Cleaning to the Hold-out Test Data ---")
try:
    # Load your latest hold-out test set
    df_holdout_test = pd.read_parquet(f'{WORKING_DIR}holdout_test_advanced_features.parquet')
    print("Successfully loaded holdout_test_advanced_features.parquet.")
    print(f"Original hold-out test data shape: {df_holdout_test.shape}")
except FileNotFoundError:
    print("Error: holdout_test_advanced_features.parquet not found.")
    exit()

# --- Drop the same columns from the test set for consistency ---
# Identify which of the columns to drop actually exist in the test set
cols_to_drop_in_test = [col for col in cols_to_drop if col in df_holdout_test.columns]
df_holdout_test_cleaned = df_holdout_test.drop(columns=cols_to_drop_in_test)
print(f"\nRemoved {len(cols_to_drop_in_test)} columns from the hold-out test set.")
print(f"New hold-out test data shape: {df_holdout_test_cleaned.shape}")


# =============================================================================
# STAGE 3: SAVE THE FINAL CLEANED DATASETS
# =============================================================================
print("\n--- STAGE 3: Saving Final Cleaned Datasets ---")
try:
    train_output_path = f'{WORKING_DIR}train_cleaned.parquet'
    holdout_test_output_path = f'{WORKING_DIR}holdout_test_cleaned.parquet'
    
    df_train_cleaned.to_parquet(train_output_path)
    df_holdout_test_cleaned.to_parquet(holdout_test_output_path)
    
    print(f"\nSuccessfully saved final training data to: {train_output_path}")
    print(f"Successfully saved final holdout-test data to: {holdout_test_output_path}")
except Exception as e:
    print(f"\nError saving the final files: {e}")

print("\n--- DATA CLEANING COMPLETE ---")


--- STAGE 1: Analyzing Training Data for Zero-Variance Features ---
Successfully loaded train_advanced_features.parquet.
Original training data shape: (166729, 393)

Found 0 columns with zero variance to remove.

New training data shape after removing zero-variance columns: (166729, 393)

--- STAGE 2: Applying the Same Cleaning to the Hold-out Test Data ---
Successfully loaded holdout_test_advanced_features.parquet.
Original hold-out test data shape: (55577, 393)

Removed 0 columns from the hold-out test set.
New hold-out test data shape: (55577, 393)

--- STAGE 3: Saving Final Cleaned Datasets ---

Successfully saved final training data to: /kaggle/working/train_cleaned.parquet
Successfully saved final holdout-test data to: /kaggle/working/holdout_test_cleaned.parquet

--- DATA CLEANING COMPLETE ---


In [10]:
import pandas as pd
import numpy as np
import warnings
import gc
import os

# Modeling libraries
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans

# Suppress warnings for cleaner output
warnings.filterwarnings('ignore')

# --- Define Kaggle File Paths ---
INPUT_DIR = '/kaggle/input/amexkkgp/'
WORKING_DIR = '/kaggle/working/'

# =============================================================================
# STAGE 1: LOAD DATA
# =============================================================================
print("--- STAGE 1: Loading Data ---")
try:
    # Load your latest cleaned training and holdout test sets
    df_train = pd.read_parquet(f'{WORKING_DIR}train_cleaned.parquet')
    df_holdout_test = pd.read_parquet(f'{WORKING_DIR}holdout_test_cleaned.parquet')
    
    # Load supplementary data for feature creation
    offer_meta_df = pd.read_parquet(f'{INPUT_DIR}offer_metadata.parquet')
    # Use pandas for these as they are needed for clustering and should fit in memory
    event_df = pd.read_parquet(f'{INPUT_DIR}add_event.parquet')
    trans_df = pd.read_parquet(f'{INPUT_DIR}add_trans.parquet')
    
    print("Successfully loaded all required data.")
    print(f"Input training data shape: {df_train.shape}")
    print(f"Input hold-out test data shape: {df_holdout_test.shape}")
    
except FileNotFoundError:
    print("Error: Required data not found. Please ensure the data_cleaning.py script ran successfully.")
    exit()

# =============================================================================
# STAGE 2: OFFER CLUSTERING
# =============================================================================
print("\n--- STAGE 2: Creating Offer Clusters ---")

# --- a. Create features for each offer from historical event data ---
print("Aggregating historical features for each offer...")
event_df['impression_time'] = pd.to_datetime(event_df['id4'], errors='coerce')
event_df['hour_of_day'] = event_df['impression_time'].dt.hour
event_df['is_weekend'] = (event_df['impression_time'].dt.dayofweek >= 5).astype(int)
event_df['clicked'] = event_df['id7'].notna().astype(int)

# Aggregate stats for each offer
offer_features = event_df.groupby('id3').agg(
    offer_historical_ctr=('clicked', 'mean'),
    offer_total_impressions=('id4', 'count'),
    avg_impression_hour=('hour_of_day', 'mean'),
    weekend_impression_ratio=('is_weekend', 'mean')
).reset_index()

# Merge with static offer metadata
offer_meta_df['id3'] = offer_meta_df['id3'].astype(str)
offer_features['id3'] = offer_features['id3'].astype(str)
offer_features = pd.merge(offer_features, offer_meta_df[['id3', 'f375', 'f376']], on='id3', how='left')
offer_features = offer_features.rename(columns={'f375': 'offer_redemption_freq', 'f376': 'offer_discount_rate'})

# --- b. Preprocess and Cluster Offers ---
print("Preprocessing and clustering offers...")
offer_feature_cols = [col for col in offer_features.columns if col != 'id3']
X_offer = offer_features[offer_feature_cols].copy()

imputer = SimpleImputer(strategy='median')
X_offer_imputed = imputer.fit_transform(X_offer)
scaler = StandardScaler()
X_offer_scaled = scaler.fit_transform(X_offer_imputed)

pca = PCA(n_components=0.95) # Explain 95% of variance
X_offer_pca = pca.fit_transform(X_offer_scaled)
print(f"Offer PCA resulted in {pca.n_components_} components.")

kmeans_offer = KMeans(n_clusters=5, random_state=42, n_init='auto')
offer_features['offer_cluster'] = kmeans_offer.fit_predict(X_offer_pca)
print("Offer clusters created successfully.")


# =============================================================================
# STAGE 3: INDUSTRY CLUSTERING
# =============================================================================
print("\n--- STAGE 3: Creating Industry Clusters ---")

# --- a. Create features for each industry from historical transaction data ---
print("Aggregating historical features for each industry...")
trans_df['f367'] = pd.to_numeric(trans_df['f367'], errors='coerce')
industry_features = trans_df.groupby('id8').agg(
    industry_avg_spend=('f367', 'mean'),
    industry_std_spend=('f367', 'std'),
    industry_total_transactions=('f367', 'count'),
    industry_unique_products=('f368', 'nunique')
).reset_index()

# --- b. Preprocess and Cluster Industries ---
print("Preprocessing and clustering industries...")
industry_feature_cols = [col for col in industry_features.columns if col != 'id8']
X_industry = industry_features[industry_feature_cols].copy()

imputer = SimpleImputer(strategy='median')
X_industry_imputed = imputer.fit_transform(X_industry)
scaler = StandardScaler()
X_industry_scaled = scaler.fit_transform(X_industry_imputed)

pca = PCA(n_components=0.95)
X_industry_pca = pca.fit_transform(X_industry_scaled)
print(f"Industry PCA resulted in {pca.n_components_} components.")

kmeans_industry = KMeans(n_clusters=5, random_state=42, n_init='auto')
industry_features['industry_cluster'] = kmeans_industry.fit_predict(X_industry_pca)
print("Industry clusters created successfully.")


# =============================================================================
# STAGE 4: MERGE NEW FEATURES AND SAVE
# =============================================================================
print("\n--- STAGE 4: Merging New Cluster Features and Saving ---")

# --- Create a mapping from offer_id to industry_cluster ---
offer_meta_df['id8'] = offer_meta_df['id8'].astype(str)
industry_features['id8'] = industry_features['id8'].astype(str)
offer_to_industry_cluster = pd.merge(offer_meta_df[['id3', 'id8']], industry_features[['id8', 'industry_cluster']], on='id8', how='left')

# --- Merge all new features into train and holdout test sets ---
def enrich_with_entity_clusters(df, offer_features, offer_to_industry_cluster):
    df['id3'] = df['id3'].astype(str)
    # Merge offer clusters
    df = pd.merge(df, offer_features[['id3', 'offer_cluster']], on='id3', how='left')
    # Merge industry clusters (via the offer_id to industry cluster mapping)
    df = pd.merge(df, offer_to_industry_cluster[['id3', 'industry_cluster']], on='id3', how='left')
    return df

print("Enriching training data...")
df_train_final = enrich_with_entity_clusters(df_train, offer_features, offer_to_industry_cluster)
print("Enriching hold-out test data...")
df_holdout_test_final = enrich_with_entity_clusters(df_holdout_test, offer_features, offer_to_industry_cluster)

# --- Save the final DataFrames ---
try:
    train_output_path = f'{WORKING_DIR}train_entity_clusters.parquet'
    holdout_test_output_path = f'{WORKING_DIR}holdout_test_entity_clusters.parquet'
    
    df_train_final.to_parquet(train_output_path)
    df_holdout_test_final.to_parquet(holdout_test_output_path)
    
    print(f"\nSuccessfully saved final training data to: {train_output_path}")
    print(f"Successfully saved final holdout-test data to: {holdout_test_output_path}")
except Exception as e:
    print(f"\nError saving the final files: {e}")

print("\n--- ENTITY CLUSTER FEATURE ENGINEERING COMPLETE ---")

--- STAGE 1: Loading Data ---
Successfully loaded all required data.
Input training data shape: (166729, 393)
Input hold-out test data shape: (55577, 393)

--- STAGE 2: Creating Offer Clusters ---
Aggregating historical features for each offer...
Preprocessing and clustering offers...
Offer PCA resulted in 6 components.
Offer clusters created successfully.

--- STAGE 3: Creating Industry Clusters ---
Aggregating historical features for each industry...
Preprocessing and clustering industries...
Industry PCA resulted in 4 components.
Industry clusters created successfully.

--- STAGE 4: Merging New Cluster Features and Saving ---
Enriching training data...
Enriching hold-out test data...

Successfully saved final training data to: /kaggle/working/train_entity_clusters.parquet
Successfully saved final holdout-test data to: /kaggle/working/holdout_test_entity_clusters.parquet

--- ENTITY CLUSTER FEATURE ENGINEERING COMPLETE ---


In [11]:
# df_train_final.shape

(166729, 395)

In [12]:
# df_train_final.head()

,id1,id2,id3,id4,id5,y,f1,f2,f3,f4,...,profile_cluster,engagement_cluster,holistic_cluster,avg_offer_time_to_click_seconds,credit_trans_count,debit_trans_count,avg_trans_hour,offer_body_length,offer_cluster,industry_cluster
0,1309455_654480_16-23_2023-11-01 05:29:18.734,1309455,654480,2023-11-01 05:29:18.734,2023-11-01,0,None,None,None,None,...,3,1,1,45.174270,13298.0,707.0,9.883684,30.0,2,1.0
1,1277975_66290365_16-23_2023-11-01 10:00:06.073,1277975,66290365,2023-11-01 10:00:06.073,2023-11-01,0,5.0,None,13.0,88.0,...,3,1,1,50.294750,9329.0,2141.0,8.055623,33.0,4,1.0
2,1756831_783808_16-23_2023-11-02 07:45:41.794,1756831,783808,2023-11-02 07:45:41.794,2023-11-02,0,None,None,None,None,...,3,0,0,31.747560,60357.0,691.0,10.632175,30.0,2,1.0
3,1839978_80893_16-23_2023-11-02 07:03:16.575,1839978,80893,2023-11-02 07:03:16.575,2023-11-02,0,48.0,25.0,22.0,29.0,...,2,1,1,28.859316,10744.0,1005.0,8.604137,30.0,1,1.0
4,1382370_331980_16-23_2023-11-03 07:18:32.167,1382370,331980,2023-11-03 07:18:32.167,2023-11-03,0,None,40.0,None,None,...,2,0,0,19.150331,11439.0,2190.0,7.712818,34.0,2,1.0


In [9]:
# import gc

# # List of common large objects we created in previous steps
# # Add any other large variables you might have created
# variables_to_delete = [
#     'df', 'df_test', 'X', 'y', 'groups', 'df_train', 'df_test_refined',
#     'df_sample', 'X_cluster', 'X_scaled', 'X_pca', 'df_cluster', 'df_train_v9', 'df_test_v9',
#     'rfm_df', 'trans_dd', 'event_dd', 'offer_meta_df', 'data_dict', 'df_train_final', 'df_test_final'
# ]

# print("--- Clearing RAM before training ---")
# for var_name in variables_to_delete:
#     if var_name in globals():
#         print(f"Deleting: {var_name}")
#         del globals()[var_name]

# # Call the garbage collector to release the memory
# gc.collect()
# print("RAM cleared successfully.")

In [1]:
# import pandas as pd
# import numpy as np
# import xgboost as xgb
# from sklearn.model_selection import GroupKFold
# from sklearn.metrics import roc_auc_score
# import warnings
# import gc
# import os

# # Suppress warnings for cleaner output
# warnings.filterwarnings('ignore')

# # --- Define Kaggle File Paths ---
# INPUT_DIR = '/kaggle/input/amexkkgp/'
# WORKING_DIR = '/kaggle/working/'
# # =============================================================================
# # STAGE 1: DATA LOADING AND PREPARATION FOR XGBRANKER
# # =============================================================================
# print("--- STAGE 1: Loading and Preparing Data for XGBRanker ---")
# try:
#     # Load your final, pre-cleaned training and test data
#     df = pd.read_parquet(f'/kaggle/input/amex-v15/train_data_v15.parquet')
#     df_test = pd.read_parquet(f'/kaggle/input/amex-v15/test_data_v15.parquet')
#     print("Successfully loaded final training and test data.")
# except FileNotFoundError:
#     print("Error: v92 data not found. Please run the final cleaning script first.")
#     exit()

--- STAGE 1: Loading and Preparing Data for XGBRanker ---
Successfully loaded final training and test data.


In [2]:
# df = df.drop(columns=['time_to_click_seconds'], errors='ignore')
# df_test = df_test.drop(columns=['time_to_click_seconds'], errors='ignore')

In [4]:
# import pandas as pd
# import numpy as np
# import xgboost as xgb
# from sklearn.model_selection import GroupKFold
# from sklearn.metrics import roc_auc_score
# import warnings
# import gc
# import os

# # Suppress warnings for cleaner output
# warnings.filterwarnings('ignore')

# # --- Define Kaggle File Paths ---
# INPUT_DIR = '/kaggle/input/amexkkgp/'
# WORKING_DIR = '/kaggle/working/'

# # =============================================================================
# # STAGE 0: DEFINE THE MAP@7 EVALUATION METRIC
# # =============================================================================
# def map_at_k(y_true, y_pred, group_ids, k=7):
#     """
#     Calculates the Mean Average Precision at k.
    
#     Args:
#         y_true (array-like): The true relevance labels (0 or 1).
#         y_pred (array-like): The predicted ranking scores.
#         group_ids (array-like): The group/query IDs for each sample.
#         k (int): The cutoff for the precision calculation.
        
#     Returns:
#         float: The Mean Average Precision @ k score.
#     """
#     df = pd.DataFrame({'group': group_ids, 'y_true': y_true, 'y_pred': y_pred})
    
#     average_precisions = []
#     for group in df['group'].unique():
#         group_df = df[df['group'] == group].sort_values('y_pred', ascending=False).reset_index(drop=True)
        
#         # Get the top k predictions
#         group_df = group_df.head(k)
        
#         if group_df['y_true'].sum() == 0:
#             average_precisions.append(0)
#             continue
            
#         relevant_hits = 0
#         precision_at_i = []
#         for i, row in group_df.iterrows():
#             if row['y_true'] == 1:
#                 relevant_hits += 1
#                 precision_at_i.append(relevant_hits / (i + 1))
        
#         if not precision_at_i:
#             average_precisions.append(0)
#         else:
#             average_precisions.append(np.mean(precision_at_i))
            
#     return np.mean(average_precisions)


# # # =============================================================================
# # # STAGE 1: DATA LOADING AND PREPARATION FOR XGBRANKER
# # # =============================================================================
# # print("--- STAGE 1: Loading and Preparing Data for XGBRanker ---")
# # try:
# #     # Load your final, pre-cleaned training and test data
# #     df = pd.read_parquet(f'{WORKING_DIR}train_data_v92.parquet')
# #     df_test = pd.read_parquet(f'{WORKING_DIR}test_data_v92.parquet')
# #     print("Successfully loaded final training (v92) and test (v92) data.")
# # except FileNotFoundError:
# #     print("Error: v92 data not found. Please run the final cleaning script first.")
# #     exit()

# # --- Prepare Feature and Target Names ---
# target_col = 'y'
# # All columns are features except for identifiers and the target
# feature_cols = [col for col in df.columns if col not in ['id1', 'id2', 'id3', 'id4', 'id5', 'y', 'id8']]
# # Identify all categorical features to be one-hot encoded
# categorical_features = [col for col in df.columns if 'cluster' in col or col == 'offer_type_code' or col == 'offer_type_x_weekend']

# # --- Preprocessing ---
# print("Preprocessing data for training...")
# # Handle one-hot encoding for categorical features
# df = pd.get_dummies(df, columns=categorical_features, dummy_na=True)
# df_test = pd.get_dummies(df_test, columns=categorical_features, dummy_na=True)

# # Update feature list after one-hot encoding
# for cat_col in categorical_features:
#     if cat_col in feature_cols:
#         feature_cols.remove(cat_col)
# dummy_cols = [col for col in df.columns if any(cat_col in col for cat_col in categorical_features)]
# feature_cols.extend(dummy_cols)
# # Remove any duplicates that might have been added
# feature_cols = list(dict.fromkeys(feature_cols)) 

# # Convert all feature columns to numeric before imputation
# print("Converting all feature columns to numeric types...")
# for col in feature_cols:
#     if col in df.columns:
#         df[col] = pd.to_numeric(df[col], errors='coerce')
#     if col in df_test.columns:
#         df_test[col] = pd.to_numeric(df_test[col], errors='coerce')

# # Final imputation for training data
# df[feature_cols] = df[feature_cols].fillna(-999)

# # --- Create Groups for XGBRanker ---
# # A "query" or "group" is a single ranking task (all offers for one customer on one day)
# df['group_id'] = df['id2'].astype(str) + '_' + df['id5'].astype(str)
# df = df.sort_values('group_id').reset_index(drop=True)
# group_sizes = df.groupby('group_id')['id1'].count().to_numpy()

# # Define features (X) and target (y)
# X = df[feature_cols]
# y = df[target_col].astype(int)
# print(f"Prepared training data with {len(feature_cols)} features and {len(group_sizes)} groups.")


# # =============================================================================
# # STAGE 2: TRAINING FINAL ENSEMBLE WITH XGBRANKER
# # =============================================================================
# print("\n--- STAGE 2: Training Final Ensemble of XGBRanker Models ---")

# # Define XGBRanker parameters. Note the 'rank:ndcg' objective.
# final_params = {
#     'objective': 'rank:ndcg', # Learning-to-rank objective
#     'eval_metric': 'ndcg@7',  # Evaluate using a metric similar to the competition
#     'use_label_encoder': False, 'seed': 42,
#     'tree_method': 'gpu_hist', 'gpu_id': 0,
#     'n_estimators': 1000, 
#     'learning_rate': 0.05, 
#     'max_depth': 7, 
#     'subsample': 0.8, 
#     'colsample_bytree': 0.8
# }

# N_SPLITS = 5 # Using 5 folds for a balance of stability and speed
# gkf_train = GroupKFold(n_splits=N_SPLITS)
# oof_predictions = np.zeros(len(df))
# oof_map_scores = []

# print(f"Starting final training with {N_SPLITS}-Fold GroupKFold on GPU...")
# # For XGBRanker, we must use the customer ID for the GroupKFold split
# groups_for_split = df['id2']

# # MODIFIED: Corrected the variable name from 'for_split' to 'groups_for_split'
# for fold, (train_idx, val_idx) in enumerate(gkf_train.split(X, y, groups=groups_for_split)):
#     print(f"--- Fold {fold+1}/{N_SPLITS} ---")
    
#     X_train_fold, y_train_fold = X.iloc[train_idx], y.iloc[train_idx]
#     X_val_fold, y_val_fold = X.iloc[val_idx], y.iloc[val_idx]
    
#     # Get the group sizes for the training and validation sets
#     train_groups = df.iloc[train_idx].groupby('group_id')['id1'].count().to_numpy()
#     val_groups = df.iloc[val_idx].groupby('group_id')['id1'].count().to_numpy()
    
#     model = xgb.XGBRanker(**final_params)
#     model.fit(X_train_fold, y_train_fold, group=train_groups,
#               eval_set=[(X_val_fold, y_val_fold)], eval_group=[val_groups],
#               early_stopping_rounds=50, verbose=False)
    
#     # XGBRanker outputs scores, not probabilities
#     val_preds = model.predict(X_val_fold)
#     oof_predictions[val_idx] = val_preds

#     # --- Calculate and store MAP@7 for this fold ---
#     fold_map_score = map_at_k(y_val_fold, val_preds, df.iloc[val_idx]['group_id'], k=7)
#     oof_map_scores.append(fold_map_score)
#     print(f"Fold {fold+1} MAP@7 Score: {fold_map_score:.5f}")

#     model_path = f'{WORKING_DIR}final_ranker_model_fold_{fold+1}.json'
#     model.save_model(model_path)
#     print(f"Model for fold {fold+1} saved to {model_path}")
#     del X_train_fold, y_train_fold, X_val_fold, y_val_fold, model
#     gc.collect()

# # --- Evaluate Overall Performance ---
# print(f"\n--- Overall Cross-Validation Results ---")
# print(f"Mean Out-of-Fold (OOF) MAP@7 Score: {np.mean(oof_map_scores):.5f}")
# print("This score is the most reliable estimate of your final leaderboard performance.")


# # =============================================================================
# # STAGE 3: PREDICTION AND SUBMISSION FILE CREATION
# # =============================================================================
# print("\n--- STAGE 3: Generating Final Predictions ---")
# # Prepare test data
# submission_ids = df_test[['id1', 'id2', 'id3', 'id5']].copy()
# # Align test columns with the training columns
# X_test = df_test.reindex(columns=X.columns, fill_value=0)
# # Final imputation for test data
# X_test = X_test.fillna(-999)

# # Load models and predict
# test_predictions = np.zeros(len(X_test))
# for fold in range(1, N_SPLITS + 1):
#     print(f"Predicting with Fold {fold}/{N_SPLITS}...")
#     model_path = f'{WORKING_DIR}final_ranker_model_fold_{fold}.json'
#     model = xgb.XGBRanker()
#     model.load_model(model_path)
#     # Predict outputs ranking scores
#     test_predictions += model.predict(X_test) / N_SPLITS

# # Create submission file
# print("\nCreating submission file...")
# submission_df = submission_ids.copy()
# submission_df['id5'] = pd.to_datetime(submission_df['id5'], errors='coerce').dt.strftime('%m-%d-%Y')
# cleaned_predictions = np.nan_to_num(test_predictions, nan=-999) # Use a low score for NaNs
# submission_df['pred'] = cleaned_predictions

# submission_path = f'{WORKING_DIR}r2_submission_final_ranker.csv'
# submission_df.to_csv(submission_path, index=False)
# print(f"\nSubmission file successfully saved to: {submission_path}")
# print("\nFirst 5 rows of the submission file:")
# print(submission_df.head())



--- STAGE 2: Training Final Ensemble of XGBRanker Models ---
Starting final training with 5-Fold GroupKFold on GPU...
--- Fold 1/5 ---
Fold 1 MAP@7 Score: 0.06066
Model for fold 1 saved to /kaggle/working/final_ranker_model_fold_1.json
--- Fold 2/5 ---
Fold 2 MAP@7 Score: 0.06649
Model for fold 2 saved to /kaggle/working/final_ranker_model_fold_2.json
--- Fold 3/5 ---
Fold 3 MAP@7 Score: 0.06375
Model for fold 3 saved to /kaggle/working/final_ranker_model_fold_3.json
--- Fold 4/5 ---
Fold 4 MAP@7 Score: 0.06472
Model for fold 4 saved to /kaggle/working/final_ranker_model_fold_4.json
--- Fold 5/5 ---
Fold 5 MAP@7 Score: 0.06350
Model for fold 5 saved to /kaggle/working/final_ranker_model_fold_5.json

--- Overall Cross-Validation Results ---
Mean Out-of-Fold (OOF) MAP@7 Score: 0.06382
This score is the most reliable estimate of your final leaderboard performance.

--- STAGE 3: Generating Final Predictions ---
Predicting with Fold 1/5...
Predicting with Fold 2/5...
Predicting with Fold 3

In [ ]:
# # =============================================================================
# # SECTION 0: SETUP AND IMPORTS
# # =============================================================================
# print("--- Section 0: Setting up the environment ---")
# # Install the necessary libraries for graph and sequential embeddings
# !pip install -q node2vec gensim

# import pandas as pd
# import numpy as np
# import dask.dataframe as dd
# import warnings
# import gc
# import os
# import networkx as nx
# from node2vec import Node2Vec
# from gensim.models import Word2Vec

# # Suppress warnings for cleaner output
# warnings.filterwarnings('ignore')

# # --- Define Kaggle File Paths ---
# INPUT_DIR = '/kaggle/input/amexkkgp/'
# WORKING_DIR = '/kaggle/working/'

# # =============================================================================
# # STAGE 1: LOAD DATA
# # =============================================================================
# print("\n--- Section 1: Loading Data ---")
# try:
#     # Load your latest training and test sets
#     df_train = pd.read_parquet(f'/kaggle/input/amex-v15/train_data_v15.parquet')
#     df_test = pd.read_parquet(f'/kaggle/input/amex-v15/test_data_v15.parquet')
    
#     # Use Dask for the large event log
#     event_dd = dd.read_parquet(f'{INPUT_DIR}add_event.parquet', columns=['id2', 'id3', 'id4'])
    
#     print("Successfully loaded all required data.")
# except FileNotFoundError:
#     print("Error: Required data not found. Please ensure all necessary files are in the correct directories.")
#     exit()

# # =============================================================================
# # TECHNIQUE 1: GRAPH EMBEDDINGS (NODEVEC)
# # =============================================================================
# print("\n--- Technique 1: Creating Graph Embedding Features ---")

# # --- Step 1a: Build the Interaction Graph ---
# print("Step 1a: Building the customer-offer interaction graph...")
# # Compute the edge list (customer, offer pairs) from the Dask DataFrame
# # This is the most memory-intensive part of this stage
# edge_list = event_dd[['id2', 'id3']].compute()
# # Create a graph from the edge list
# G = nx.from_pandas_edgelist(edge_list, 'id2', 'id3')
# print(f"Graph created with {G.number_of_nodes()} nodes and {G.number_of_edges()} edges.")

# # --- Step 1b: Train the Node2Vec Model ---
# print("Step 1b: Training the Node2Vec model (this will take a long time)...")
# # Initialize Node2Vec. Parameters can be tuned.
# node2vec = Node2Vec(G, dimensions=32, walk_length=20, num_walks=10, workers=4, quiet=True)
# # Train the model
# model_n2v = node2vec.fit(window=5, min_count=1, batch_words=4)
# print("Node2Vec model trained successfully.")

# # --- Step 1c: Extract Offer Embeddings ---
# print("Step 1c: Extracting offer embeddings...")
# offer_ids = df_train['id3'].unique().tolist() + df_test['id3'].unique().tolist()
# offer_ids = list(set(offer_ids)) # Get unique offer IDs across train and test

# graph_embeddings = []
# for offer_id in offer_ids:
#     try:
#         vector = model_n2v.wv[offer_id]
#         graph_embeddings.append([offer_id] + vector.tolist())
#     except KeyError:
#         # This offer was not in the historical graph
#         continue

# graph_embedding_features = pd.DataFrame(graph_embeddings, columns=['id3'] + [f'graph_emb_{i}' for i in range(32)])
# print("Graph embedding features created successfully.")

# # Clean up memory
# del G, edge_list, model_n2v, node2vec
# gc.collect()

# # =============================================================================
# # TECHNIQUE 2: SEQUENTIAL EMBEDDINGS (WORD2VEC)
# # =============================================================================
# print("\n--- Technique 2: Creating Sequential Embedding Features ---")

# # --- Step 2a: Create Customer Sessions ---
# print("Step 2a: Creating customer sessions...")
# event_dd['impression_time'] = dd.to_datetime(event_dd['id4'], errors='coerce')

# # MODIFIED: Use a robust, Dask-native method to create time-based sessions
# # Sort by customer and time
# event_dd = event_dd.set_index('impression_time').sort_index()
# event_dd = event_dd.reset_index()
# event_dd = event_dd.set_index('id2').sort_index()

# # Calculate time difference between consecutive events for each customer
# event_dd['time_diff'] = event_dd.groupby('id2')['impression_time'].diff().dt.total_seconds()

# # A new session starts if the time difference is > 30 minutes (1800 seconds)
# event_dd['new_session'] = (event_dd['time_diff'] > 1800).fillna(True)
# event_dd['session_id'] = event_dd.groupby('id2')['new_session'].cumsum()

# # Create the list of sessions (sequences of offers)
# sessions = event_dd.groupby(['id2', 'session_id'])['id3'].apply(list, meta=('id3', 'object')).compute().tolist()
# print(f"Created {len(sessions)} customer sessions.")

# # --- Step 2b: Train the Word2Vec Model ---
# print("Step 2b: Training the Word2Vec model...")
# # Train the model on the sessions. Treats offers as "words" and sessions as "sentences".
# model_w2v = Word2Vec(sentences=sessions, vector_size=32, window=5, min_count=1, workers=4)
# print("Word2Vec model trained successfully.")

# # --- Step 2c: Extract Offer Embeddings ---
# print("Step 2c: Extracting sequential embeddings...")
# sequential_embeddings = []
# for offer_id in offer_ids:
#     try:
#         vector = model_w2v.wv[offer_id]
#         sequential_embeddings.append([offer_id] + vector.tolist())
#     except KeyError:
#         continue

# sequential_embedding_features = pd.DataFrame(sequential_embeddings, columns=['id3'] + [f'seq_emb_{i}' for i in range(32)])
# print("Sequential embedding features created successfully.")

# # Clean up memory
# del sessions, model_w2v
# gc.collect()

# # =============================================================================
# # STAGE 3: MERGE NEW FEATURES AND SAVE
# # =============================================================================
# print("\n--- STAGE 3: Merging New Features and Saving Final Datasets (v11) ---")

# def enrich_dataframe(df, graph_embedding_features, sequential_embedding_features):
#     """A reusable function to enrich a dataframe with new embedding features."""
#     df['id3'] = df['id3'].astype(str)
#     graph_embedding_features['id3'] = graph_embedding_features['id3'].astype(str)
#     sequential_embedding_features['id3'] = sequential_embedding_features['id3'].astype(str)
    
#     # Merge graph embedding features
#     df = pd.merge(df, graph_embedding_features, on='id3', how='left')
#     # Merge sequential embedding features
#     df = pd.merge(df, sequential_embedding_features, on='id3', how='left')
    
#     return df

# # --- Enrich both the training and test dataframes ---
# print("Enriching training data...")
# df_train_final = enrich_dataframe(df_train, graph_embedding_features, sequential_embedding_features)
# print("Enriching test data...")
# df_test_final = enrich_dataframe(df_test, graph_embedding_features, sequential_embedding_features)

# # --- Save the final DataFrames ---
# try:
#     train_output_path = f'{WORKING_DIR}train_data_v16.parquet'
#     test_output_path = f'{WORKING_DIR}test_data_v16.parquet'
    
#     df_train_final.to_parquet(train_output_path)
#     df_test_final.to_parquet(test_output_path)
    
#     print(f"\nSuccessfully saved final training data to: {train_output_path}")
#     print(f"Successfully saved final test data to: {test_output_path}")
# except Exception as e:
#     print(f"\nError saving the final files: {e}")

# print("\n--- ADVANCED EMBEDDING FEATURE ENGINEERING COMPLETE ---")

--- Section 0: Setting up the environment ---
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.6/38.6 MB 51.7 MB/s eta 0:00:00:00:0100:01
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cesium 0.12.4 requires numpy<3.0,>=2.0, but you have numpy 1.26.4 which is incompatible.
tsfresh 0.21.0 requires scipy>=1.14.0; python_version >= "3.10", but you have scipy 1.13.1 which is incompatible.
dopamine-rl 4.1.2 requires gymnasium>=1.0.0, but you have gymnasium 0.29.0 which is incompatible.
imbalanced-learn 0.13.0 requires scikit-learn<2,>=1.3.2, but you have scikit-learn 1.2.2 which is incompatible.
plotnine 0.14.5 requires matplotlib>=3.8.0, but you have matplotlib 3.7.2 which is incompatible.
mlxtend 0.23.4 requires scikit-learn>=1.3.1, but you have scikit-learn 1.2.2 which is incompatible.

In [1]:
# # =============================================================================
# # SECTION 0: SETUP AND IMPORTS
# # =============================================================================
# print("--- Section 0: Setting up the environment ---")
# # Install the necessary library for text embeddings
# !pip install -q sentence-transformers

# import pandas as pd
# import numpy as np
# import dask.dataframe as dd
# import warnings
# import gc
# import os

# from sentence_transformers import SentenceTransformer

# # Suppress warnings for cleaner output
# warnings.filterwarnings('ignore')

# # --- Define Kaggle File Paths ---
# INPUT_DIR = '/kaggle/input/amexkkgp/'
# WORKING_DIR = '/kaggle/working/'

# # =============================================================================
# # STAGE 1: LOAD DATA
# # =============================================================================
# print("\n--- Section 1: Loading Data ---")
# try:
#     # Load your latest training and test sets
#     df_train = pd.read_parquet(f'/kaggle/input/amex-v15/train_data_v15.parquet')
#     df_test = pd.read_parquet(f'/kaggle/input/amex-v15/test_data_v15.parquet')
    
#     # Load supplementary data for feature creation
#     offer_meta_df = pd.read_parquet(f'{INPUT_DIR}offer_metadata.parquet')
#     event_dd = dd.read_parquet(f'{INPUT_DIR}add_event.parquet')
    
#     print("Successfully loaded all required data.")
# except FileNotFoundError:
#     print("Error: Required data not found. Please ensure all necessary files are in the correct directories.")
#     exit()

# # =============================================================================
# # TECHNIQUE 1: CO-CLICK & SEQUENTIAL FEATURES (MEMORY-EFFICIENT)
# # =============================================================================
# print("\n--- Technique 1: Creating Co-Click Features ---")

# # --- Step 1a: Create Customer Sessions on a Sample ---
# print("Step 1a: Creating customer sessions on a sample of the data...")
# # Use a large sample of the event data to make the process much faster
# event_dd_sample = event_dd.sample(frac=0.15, random_state=42)
# event_dd_sample['impression_time'] = dd.to_datetime(event_dd_sample['id4'], errors='coerce')

# # Use a robust, Dask-native method to create time-based sessions
# print("Sorting events by customer and time...")
# event_dd_sample = event_dd_sample.sort_values(['id2', 'impression_time'])

# def calculate_time_diff(df):
#     df['time_diff'] = df.groupby('id2')['impression_time'].diff().dt.total_seconds()
#     return df

# meta = event_dd_sample._meta.copy()
# meta['time_diff'] = pd.Series(dtype='float64')
# event_dd_sample = event_dd_sample.map_partitions(calculate_time_diff, meta=meta)

# event_dd_sample['new_session'] = (event_dd_sample['time_diff'] > 1800).fillna(True)
# event_dd_sample['session_id'] = event_dd_sample.groupby('id2')['new_session'].cumsum()
# print("Customer sessions created.")


# # --- Step 1b: Calculate Co-occurrence (Fully with Dask) ---
# print("Step 1b: Calculating offer co-occurrence within sessions using Dask...")
# # MODIFIED: Perform the entire co-occurrence calculation within Dask's lazy framework
# session_offers_dd = event_dd_sample[['session_id', 'id2', 'id3']]
# # Perform a Dask self-merge
# merged_sessions_dd = dd.merge(session_offers_dd, session_offers_dd, on=['session_id', 'id2'])
# # Filter out self-pairs
# co_occurrence_dd = merged_sessions_dd[merged_sessions_dd['id3_x'] != merged_sessions_dd['id3_y']]
# # Count how many times each pair appears and compute the final small result
# co_occurrence_counts = co_occurrence_dd.groupby(['id3_x', 'id3_y']).size().compute().reset_index(name='count')
# print("Co-occurrence calculated successfully.")


--- Section 0: Setting up the environment ---


2025-07-20 14:18:58.503146: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1753021138.720674     210 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1753021138.784423     210 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered



--- Section 1: Loading Data ---
Successfully loaded all required data.

--- Technique 1: Creating Co-Click Features ---
Step 1a: Creating customer sessions on a sample of the data...
Sorting events by customer and time...
Customer sessions created.
Step 1b: Calculating offer co-occurrence within sessions using Dask...
Co-occurrence calculated successfully.
Step 1c: Creating final co-click features...


AttributeError: 'Series' object has no attribute 'notna'

In [2]:
# # --- Step 1c: Create Co-Click Features ---
# print("Step 1c: Creating final co-click features...")
# # MODIFIED: Changed .notna() to the Dask-compatible ~.isnull() to fix the AttributeError
# event_dd['clicked'] = (~event_dd['id7'].isnull()).astype(int)
# offer_ctr = event_dd.groupby('id3')['clicked'].mean().compute().to_dict()

# # For each offer, find its most frequent partner
# co_occurrence_counts = co_occurrence_counts.sort_values('count', ascending=False)
# most_frequent_partner = co_occurrence_counts.drop_duplicates(subset=['id3_x'], keep='first')

# # Get the CTR of that most frequent partner
# most_frequent_partner['partner_ctr'] = most_frequent_partner['id3_y'].map(offer_ctr)
# co_click_features = most_frequent_partner[['id3_x', 'partner_ctr']].rename(columns={'id3_x': 'id3', 'partner_ctr': 'most_frequent_partner_ctr'})
# print("Co-click features created successfully.")

# # Clean up memory
# del event_dd, event_dd_sample, co_occurrence_counts
# gc.collect()

# # =============================================================================
# # TECHNIQUE 2: TEXT EMBEDDINGS
# # =============================================================================
# print("\n--- Technique 2: Creating Text Embedding Features ---")

# # --- Step 2a: Prepare the Text Corpus ---
# print("Step 2a: Preparing text corpus from offer metadata...")
# offer_meta_df['id3'] = offer_meta_df['id3'].astype(str)
# # Combine text fields, handling missing values
# offer_meta_df['full_text'] = offer_meta_df['id9'].fillna('') + ' ' + offer_meta_df['f378'].fillna('')
# offer_texts = offer_meta_df[['id3', 'full_text']].copy()
# print("Text corpus prepared.")

# # --- Step 2b: Generate Embeddings ---
# print("Step 2b: Generating text embeddings (this may take a moment)...")
# # Use a lightweight but powerful pre-trained model
# model = SentenceTransformer('all-MiniLM-L6-v2')
# # Generate the embeddings
# embeddings = model.encode(offer_texts['full_text'].tolist(), show_progress_bar=True)
# print(f"Embeddings generated with shape: {embeddings.shape}")

# # Create a DataFrame from the embeddings
# embedding_df = pd.DataFrame(embeddings, columns=[f'text_emb_{i}' for i in range(embeddings.shape[1])])
# text_embedding_features = pd.concat([offer_texts[['id3']], embedding_df], axis=1)
# print("Text embedding features created successfully.")


# # =============================================================================
# # STAGE 3: MERGE NEW FEATURES AND SAVE
# # =============================================================================
# print("\n--- STAGE 3: Merging New Features and Saving Final Datasets (v10) ---")

# def enrich_dataframe(df, co_click_features, text_embedding_features):
#     """A reusable function to enrich a dataframe with new features."""
#     df['id3'] = df['id3'].astype(str)
#     co_click_features['id3'] = co_click_features['id3'].astype(str)
#     text_embedding_features['id3'] = text_embedding_features['id3'].astype(str)
    
#     # Merge co-click features
#     df = pd.merge(df, co_click_features, on='id3', how='left')
#     # Merge text embedding features
#     df = pd.merge(df, text_embedding_features, on='id3', how='left')
    
#     return df

# # --- Enrich both the training and test dataframes ---
# print("Enriching training data...")
# df_train_final = enrich_dataframe(df_train, co_click_features, text_embedding_features)
# print("Enriching test data...")
# df_test_final = enrich_dataframe(df_test, co_click_features, text_embedding_features)

# # --- Save the final DataFrames ---
# try:
#     train_output_path = f'{WORKING_DIR}train_data_v16.parquet'
#     test_output_path = f'{WORKING_DIR}test_data_v16.parquet'
    
#     df_train_final.to_parquet(train_output_path)
#     df_test_final.to_parquet(test_output_path)
    
#     print(f"\nSuccessfully saved final training data to: {train_output_path}")
#     print(f"Successfully saved final test data to: {test_output_path}")
# except Exception as e:
#     print(f"\nError saving the final files: {e}")

# print("\n--- ADVANCED FEATURE ENGINEERING COMPLETE ---")

Step 1c: Creating final co-click features...
Co-click features created successfully.

--- Technique 2: Creating Text Embedding Features ---
Step 2a: Preparing text corpus from offer metadata...
Text corpus prepared.
Step 2b: Generating text embeddings (this may take a moment)...


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/131 [00:00<?, ?it/s]

Embeddings generated with shape: (4164, 384)
Text embedding features created successfully.

--- STAGE 3: Merging New Features and Saving Final Datasets (v10) ---
Enriching training data...
Enriching test data...

Successfully saved final training data to: /kaggle/working/train_data_v16.parquet
Successfully saved final test data to: /kaggle/working/test_data_v16.parquet

--- ADVANCED FEATURE ENGINEERING COMPLETE ---


In [4]:
# df_train_final.shape

(770164, 788)

In [2]:
# import gc

# # List of large DataFrames and variables created in the previous script
# variables_to_delete = [
#     'df_train', 'df_test', 'offer_meta_df', 'event_dd', 
#     'event_dd_sample', 'session_offers', 'merged_sessions', 
#     'co_occurrence', 'co_occurrence_counts', 'event_df_pd', 
#     'offer_ctr', 'most_frequent_partner', 'co_click_features',
#     'offer_texts', 'embeddings', 'embedding_df', 
#     'text_embedding_features', 'df_train_final', 'df_test_final'
# ]

# print("--- Clearing RAM ---")
# for var_name in variables_to_delete:
#     # Check if the variable exists in the notebook's memory before trying to delete it
#     if var_name in globals():
#         print(f"Deleting: {var_name}")
#         del globals()[var_name]

# # Call the garbage collector to release the memory
# gc.collect()
# print("RAM cleared successfully.")

--- Clearing RAM ---
Deleting: df_train
Deleting: df_test
Deleting: offer_meta_df
Deleting: offer_ctr
Deleting: most_frequent_partner
Deleting: co_click_features
Deleting: offer_texts
Deleting: embeddings
Deleting: df_train_final
Deleting: df_test_final
RAM cleared successfully.


In [1]:
# # =============================================================================
# # SECTION 0: SETUP AND IMPORTS
# # =============================================================================
# print("--- Section 0: Setting up the environment ---")
# # Install the necessary library for text embeddings
# !pip install -q sentence-transformers

# import pandas as pd
# import numpy as np
# import dask.dataframe as dd
# import warnings
# import gc
# import os

# from sentence_transformers import SentenceTransformer
# from sklearn.metrics.pairwise import cosine_similarity

# # Suppress warnings for cleaner output
# warnings.filterwarnings('ignore')

# # --- Define Kaggle File Paths ---
# INPUT_DIR = '/kaggle/input/amexkkgp/'
# WORKING_DIR = '/kaggle/working/'

# # =============================================================================
# # STAGE 1: LOAD DATA
# # =============================================================================
# print("\n--- Section 1: Loading Data ---")
# try:
#     # Load your latest training and test sets
#     df_train = pd.read_parquet(f'/kaggle/input/amex-v15/train_data_v15.parquet')
#     df_test = pd.read_parquet(f'/kaggle/input/amex-v15/test_data_v15.parquet')
    
#     # Load supplementary data for feature creation
#     offer_meta_df = pd.read_parquet(f'{INPUT_DIR}offer_metadata.parquet')
#     event_dd = dd.read_parquet(f'{INPUT_DIR}add_event.parquet')
    
#     print("Successfully loaded all required data.")
# except FileNotFoundError:
#     print("Error: Required data not found. Please ensure all necessary files are in the correct directories.")
#     exit()

# # =============================================================================
# # TECHNIQUE 1: ADVANCED CO-CLICK FEATURES
# # =============================================================================
# print("\n--- Technique 1: Creating Advanced Co-Click Features ---")

# # --- Step 1a: Create Customer Sessions on a Sample ---
# print("Step 1a: Creating customer sessions on a sample of the data...")
# event_dd_sample = event_dd.sample(frac=0.2, random_state=42)
# event_dd_sample['impression_time'] = dd.to_datetime(event_dd_sample['id4'], errors='coerce')
# event_dd_sample = event_dd_sample.sort_values(['id2', 'impression_time'])

# def calculate_time_diff(df):
#     df['time_diff'] = df.groupby('id2')['impression_time'].diff().dt.total_seconds()
#     return df

# meta = event_dd_sample._meta.copy()
# meta['time_diff'] = pd.Series(dtype='float64')
# event_dd_sample = event_dd_sample.map_partitions(calculate_time_diff, meta=meta)

# event_dd_sample['new_session'] = (event_dd_sample['time_diff'] > 1800).fillna(True)
# event_dd_sample['session_id'] = event_dd_sample.groupby('id2')['new_session'].cumsum()
# print("Customer sessions created.")


# # --- Step 1b: Calculate Co-occurrence ---
# print("Step 1b: Calculating offer co-occurrence within sessions...")
# session_offers_dd = event_dd_sample[['session_id', 'id2', 'id3']]
# merged_sessions_dd = dd.merge(session_offers_dd, session_offers_dd, on=['session_id', 'id2'])
# co_occurrence_dd = merged_sessions_dd[merged_sessions_dd['id3_x'] != merged_sessions_dd['id3_y']]
# co_occurrence_counts = co_occurrence_dd.groupby(['id3_x', 'id3_y']).size().compute().reset_index(name='count')
# print("Co-occurrence calculated successfully.")


# # --- Step 1c: Create Advanced Co-Click Features ---
# print("Step 1c: Creating final co-click features...")
# event_dd['clicked'] = (~event_dd['id7'].isnull()).astype(int)
# offer_ctr = event_dd.groupby('id3')['clicked'].mean().compute().to_dict()

# # Get the CTR of each partner offer
# co_occurrence_counts['partner_ctr'] = co_occurrence_counts['id3_y'].map(offer_ctr)
# co_occurrence_counts = co_occurrence_counts.sort_values(['id3_x', 'count'], ascending=[True, False])

# # Find the top 5 partners for each offer
# top_5_partners = co_occurrence_counts.groupby('id3_x').head(5)

# # Calculate aggregate stats for the top 5 partners' CTRs
# co_click_agg = top_5_partners.groupby('id3_x')['partner_ctr'].agg(['mean', 'max', 'std']).reset_index()
# co_click_features = co_click_agg.rename(columns={
#     'id3_x': 'id3',
#     'mean': 'top5_partner_mean_ctr',
#     'max': 'top5_partner_max_ctr',
#     'std': 'top5_partner_std_ctr'
# })
# print("Advanced co-click features created successfully.")

# # Clean up memory
# del event_dd, event_dd_sample, co_occurrence_counts
# gc.collect()

# # =============================================================================
# # TECHNIQUE 2: TEXT SIMILARITY FEATURES
# # =============================================================================
# print("\n--- Technique 2: Creating Text Similarity Features ---")

# # --- Step 2a: Prepare the Text Corpus and Generate Embeddings ---
# print("Step 2a: Preparing text corpus and generating embeddings...")
# offer_meta_df['id3'] = offer_meta_df['id3'].astype(str)
# offer_meta_df['full_text'] = offer_meta_df['id9'].fillna('') + ' ' + offer_meta_df['f378'].fillna('')
# offer_texts = offer_meta_df[['id3', 'id8', 'full_text']].copy()

# model = SentenceTransformer('all-MiniLM-L6-v2')
# embeddings = model.encode(offer_texts['full_text'].tolist(), show_progress_bar=True)
# embedding_df = pd.DataFrame(embeddings, columns=[f'text_emb_{i}' for i in range(embeddings.shape[1])])
# offer_embeddings_df = pd.concat([offer_texts[['id3', 'id8']], embedding_df], axis=1)
# print("Text embeddings generated.")

# # --- Step 2b: Create Industry DNA and Calculate Similarity ---
# print("Step 2b: Calculating offer-industry text similarity...")
# # Calculate the average embedding vector for each industry
# industry_dna = offer_embeddings_df.groupby('id8').mean(numeric_only=True).reset_index()
# industry_dna = industry_dna.rename(columns={col: f'industry_{col}' for col in industry_dna.columns if 'emb' in col})

# # Merge the industry DNA back to each offer
# offer_embeddings_df = pd.merge(offer_embeddings_df, industry_dna, on='id8', how='left')

# # Calculate the cosine similarity between each offer's embedding and its industry's average embedding
# emb_cols = [f'text_emb_{i}' for i in range(embeddings.shape[1])]
# industry_emb_cols = [f'industry_text_emb_{i}' for i in range(embeddings.shape[1])]

# # This is a memory-intensive step, so we process in chunks if needed
# # For now, we attempt it directly as the dataframe is moderately sized
# offer_embeddings_df['offer_industry_text_similarity'] = [
#     cosine_similarity(row[emb_cols].values.reshape(1, -1), row[industry_emb_cols].values.reshape(1, -1))[0][0]
#     if not row[industry_emb_cols].isnull().any() else np.nan
#     for index, row in offer_embeddings_df.iterrows()
# ]
# text_similarity_features = offer_embeddings_df[['id3', 'offer_industry_text_similarity']]
# print("Text similarity features created successfully.")


# # =============================================================================
# # STAGE 3: MERGE NEW FEATURES AND SAVE
# # =============================================================================
# print("\n--- STAGE 3: Merging New Features and Saving Final Datasets (v17) ---")

# def enrich_dataframe(df, co_click_features, text_similarity_features):
#     """A reusable function to enrich a dataframe with new features."""
#     df['id3'] = df['id3'].astype(str)
#     co_click_features['id3'] = co_click_features['id3'].astype(str)
#     text_similarity_features['id3'] = text_similarity_features['id3'].astype(str)
    
#     # Merge co-click features
#     df = pd.merge(df, co_click_features, on='id3', how='left')
#     # Merge text similarity features
#     df = pd.merge(df, text_similarity_features, on='id3', how='left')
    
#     return df

# # --- Enrich both the training and test dataframes ---
# print("Enriching training data...")
# df_train_final = enrich_dataframe(df_train, co_click_features, text_similarity_features)
# print("Enriching test data...")
# df_test_final = enrich_dataframe(df_test, co_click_features, text_similarity_features)

# # --- Save the final DataFrames ---
# try:
#     train_output_path = f'{WORKING_DIR}train_data_v18.parquet'
#     test_output_path = f'{WORKING_DIR}test_data_v18.parquet'
    
#     df_train_final.to_parquet(train_output_path)
#     df_test_final.to_parquet(test_output_path)
    
#     print(f"\nSuccessfully saved final training data to: {train_output_path}")
#     print(f"Successfully saved final test data to: {test_output_path}")
# except Exception as e:
#     print(f"\nError saving the final files: {e}")

# print("\n--- ADVANCED FEATURE ENGINEERING COMPLETE ---")


--- Section 0: Setting up the environment ---
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 99.0 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 76.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 41.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 31.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.7 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 3.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 87.0 MB/s eta 0:00:00:00:0100:01


2025-07-20 16:48:57.174338: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1753030137.348404      79 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1753030137.396031      79 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered



--- Section 1: Loading Data ---
Successfully loaded all required data.

--- Technique 1: Creating Advanced Co-Click Features ---
Step 1a: Creating customer sessions on a sample of the data...
Customer sessions created.
Step 1b: Calculating offer co-occurrence within sessions...
Co-occurrence calculated successfully.
Step 1c: Creating final co-click features...
Advanced co-click features created successfully.

--- Technique 2: Creating Text Similarity Features ---
Step 2a: Preparing text corpus and generating embeddings...


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/131 [00:00<?, ?it/s]

Text embeddings generated.
Step 2b: Calculating offer-industry text similarity...
Text similarity features created successfully.

--- STAGE 3: Merging New Features and Saving Final Datasets (v17) ---
Enriching training data...
Enriching test data...

Successfully saved final training data to: /kaggle/working/train_data_v18.parquet
Successfully saved final test data to: /kaggle/working/test_data_v18.parquet

--- ADVANCED FEATURE ENGINEERING COMPLETE ---


In [ ]:
df_train_final.head()

In [2]:
# import pandas as pd
# import numpy as np
# import xgboost as xgb
# from sklearn.model_selection import GroupKFold
# from sklearn.metrics import roc_auc_score
# import warnings
# import gc
# import os

# # Suppress warnings for cleaner output
# warnings.filterwarnings('ignore')

# # --- Define Kaggle File Paths ---
# INPUT_DIR = '/kaggle/input/amexkkgp/'
# WORKING_DIR = '/kaggle/working/'

# # =============================================================================
# # STAGE 0: DEFINE THE MAP@7 EVALUATION METRIC
# # =============================================================================
# def map_at_k(y_true, y_pred, group_ids, k=7):
#     """
#     Calculates the Mean Average Precision at k.
    
#     Args:
#         y_true (array-like): The true relevance labels (0 or 1).
#         y_pred (array-like): The predicted ranking scores.
#         group_ids (array-like): The group/query IDs for each sample.
#         k (int): The cutoff for the precision calculation.
        
#     Returns:
#         float: The Mean Average Precision @ k score.
#     """
#     df = pd.DataFrame({'group': group_ids, 'y_true': y_true, 'y_pred': y_pred})
    
#     average_precisions = []
#     for group in df['group'].unique():
#         group_df = df[df['group'] == group].sort_values('y_pred', ascending=False).reset_index(drop=True)
        
#         # Get the top k predictions
#         group_df = group_df.head(k)
        
#         if group_df['y_true'].sum() == 0:
#             average_precisions.append(0)
#             continue
            
#         relevant_hits = 0
#         precision_at_i = []
#         for i, row in group_df.iterrows():
#             if row['y_true'] == 1:
#                 relevant_hits += 1
#                 precision_at_i.append(relevant_hits / (i + 1))
        
#         if not precision_at_i:
#             average_precisions.append(0)
#         else:
#             average_precisions.append(np.mean(precision_at_i))
            
#     return np.mean(average_precisions)


# # =============================================================================
# # STAGE 1: DATA LOADING AND PREPARATION FOR XGBRANKER
# # =============================================================================
# print("--- STAGE 1: Loading and Preparing Data for XGBRanker ---")
# try:
#     # Load your final, pre-cleaned training and test data
#     df = pd.read_parquet(f'{WORKING_DIR}train_data_v18.parquet')
#     df_test = pd.read_parquet(f'{WORKING_DIR}test_data_v18.parquet')
#     print("Successfully loaded final training (v92) and test (v92) data.")
# except FileNotFoundError:
#     print("Error: v92 data not found. Please run the final cleaning script first.")
#     exit()

# # --- Memory Reduction Function ---
# def reduce_mem_usage(df):
#     """Iterate through all the columns of a dataframe and modify the data type to reduce memory usage."""
#     start_mem = df.memory_usage().sum() / 1024**2
#     print(f'Memory usage of dataframe is {start_mem:.2f} MB')
    
#     for col in df.columns:
#         col_type = df[col].dtype
#         if str(col_type)[:3] == 'int' or str(col_type)[:5] == 'float':
#             c_min = df[col].min()
#             c_max = df[col].max()
#             if str(col_type)[:3] == 'int':
#                 if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
#                     df[col] = df[col].astype(np.int8)
#                 elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
#                     df[col] = df[col].astype(np.int16)
#                 elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
#                     df[col] = df[col].astype(np.int32)
#                 elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
#                     df[col] = df[col].astype(np.int64)
#             else:
#                 if c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
#                     df[col] = df[col].astype(np.float32)
#                 else:
#                     df[col] = df[col].astype(np.float64)
    
#     end_mem = df.memory_usage().sum() / 1024**2
#     print(f'Memory usage after optimization is: {end_mem:.2f} MB')
#     print(f'Decreased by {100 * (start_mem - end_mem) / start_mem:.1f}%')
#     return df

# # --- Prepare Feature and Target Names ---
# target_col = 'y'
# # All columns are features except for identifiers and the target
# feature_cols = [col for col in df.columns if col not in ['id1', 'id2', 'id3', 'id4', 'id5', 'y', 'id8']]
# # Identify all categorical features to be one-hot encoded
# categorical_features = [col for col in df.columns if 'cluster' in col or col == 'offer_type_code' or col == 'offer_type_x_weekend']

# # --- Preprocessing ---
# print("Preprocessing data for training...")
# # Handle one-hot encoding for categorical features
# df = pd.get_dummies(df, columns=categorical_features, dummy_na=True)
# df_test = pd.get_dummies(df_test, columns=categorical_features, dummy_na=True)

# # Update feature list after one-hot encoding
# for cat_col in categorical_features:
#     if cat_col in feature_cols:
#         feature_cols.remove(cat_col)
# dummy_cols = [col for col in df.columns if any(cat_col in col for cat_col in categorical_features)]
# feature_cols.extend(dummy_cols)
# # Remove any duplicates that might have been added
# feature_cols = list(dict.fromkeys(feature_cols)) 

# # Convert all feature columns to numeric before imputation
# print("Converting all feature columns to numeric types...")
# for col in feature_cols:
#     if col in df.columns:
#         df[col] = pd.to_numeric(df[col], errors='coerce')
#     if col in df_test.columns:
#         df_test[col] = pd.to_numeric(df_test[col], errors='coerce')

# # Final imputation for training data
# df[feature_cols] = df[feature_cols].fillna(-999)

# # MODIFIED: Move memory reduction to AFTER all preprocessing
# print("\nOptimizing memory usage for training data...")
# df = reduce_mem_usage(df)
# print("\nOptimizing memory usage for test data...")
# df_test = reduce_mem_usage(df_test)

# # --- Create Groups for XGBRanker ---
# # A "query" or "group" is a single ranking task (all offers for one customer on one day)
# df['group_id'] = df['id2'].astype(str) + '_' + df['id5'].astype(str)
# df = df.sort_values('group_id').reset_index(drop=True)
# group_sizes = df.groupby('group_id')['id1'].count().to_numpy()

# # Define features (X) and target (y)
# X = df[feature_cols]
# y = df[target_col].astype(int)
# print(f"Prepared training data with {len(feature_cols)} features and {len(group_sizes)} groups.")


# # =============================================================================
# # STAGE 2: TRAINING FINAL ENSEMBLE WITH XGBRANKER
# # =============================================================================
# print("\n--- STAGE 2: Training Final Ensemble of XGBRanker Models ---")

# # Define XGBRanker parameters. Note the 'rank:ndcg' objective.
# final_params = {
#     'objective': 'rank:ndcg', # Learning-to-rank objective
#     'eval_metric': 'ndcg@7',  # Evaluate using a metric similar to the competition
#     'use_label_encoder': False, 'seed': 42,
#     'tree_method': 'gpu_hist', 'gpu_id': 0,
#     'n_estimators': 2000, 
#     'learning_rate': 0.05, 
#     'max_depth': 8, 
#     'subsample': 0.8, 
#     'colsample_bytree': 0.8,
#     'max_bin': 128 # MODIFIED: Add max_bin to reduce memory usage during training
# }

# N_SPLITS = 3 # Using 5 folds for a balance of stability and speed
# gkf_train = GroupKFold(n_splits=N_SPLITS)
# oof_predictions = np.zeros(len(df))
# oof_map_scores = []

# print(f"Starting final training with {N_SPLITS}-Fold GroupKFold on GPU...")
# # For XGBRanker, we must use the customer ID for the GroupKFold split
# groups_for_split = df['id2']

# for fold, (train_idx, val_idx) in enumerate(gkf_train.split(X, y, groups=groups_for_split)):
#     print(f"--- Fold {fold+1}/{N_SPLITS} ---")
    
#     X_train_fold, y_train_fold = X.iloc[train_idx], y.iloc[train_idx]
#     X_val_fold, y_val_fold = X.iloc[val_idx], y.iloc[val_idx]
    
#     # Get the group sizes for the training and validation sets
#     train_groups = df.iloc[train_idx].groupby('group_id')['id1'].count().to_numpy()
#     val_groups = df.iloc[val_idx].groupby('group_id')['id1'].count().to_numpy()
    
#     model = xgb.XGBRanker(**final_params)
#     model.fit(X_train_fold, y_train_fold, group=train_groups,
#               eval_set=[(X_val_fold, y_val_fold)], eval_group=[val_groups],
#               early_stopping_rounds=50, verbose=False)
    
#     # XGBRanker outputs scores, not probabilities
#     val_preds = model.predict(X_val_fold)
#     oof_predictions[val_idx] = val_preds

#     # --- Calculate and store MAP@7 for this fold ---
#     fold_map_score = map_at_k(y_val_fold, val_preds, df.iloc[val_idx]['group_id'], k=7)
#     oof_map_scores.append(fold_map_score)
#     print(f"Fold {fold+1} MAP@7 Score: {fold_map_score:.5f}")

#     model_path = f'{WORKING_DIR}final_ranker_model_fold_{fold+1}.json'
#     model.save_model(model_path)
#     print(f"Model for fold {fold+1} saved to {model_path}")
#     del X_train_fold, y_train_fold, X_val_fold, y_val_fold, model
#     gc.collect()

# # --- Evaluate Overall Performance ---
# print(f"\n--- Overall Cross-Validation Results ---")
# print(f"Mean Out-of-Fold (OOF) MAP@7 Score: {np.mean(oof_map_scores):.5f}")
# print("This score is the most reliable estimate of your final leaderboard performance.")


# # =============================================================================
# # STAGE 3: PREDICTION AND SUBMISSION FILE CREATION
# # =============================================================================
# print("\n--- STAGE 3: Generating Final Predictions ---")
# # Prepare test data
# submission_ids = df_test[['id1', 'id2', 'id3', 'id5']].copy()
# # Align test columns with the training columns
# X_test = df_test.reindex(columns=X.columns, fill_value=0)
# # Final imputation for test data
# X_test = X_test.fillna(-999)

# # Load models and predict
# test_predictions = np.zeros(len(X_test))
# for fold in range(1, N_SPLITS + 1):
#     print(f"Predicting with Fold {fold}/{N_SPLITS}...")
#     model_path = f'{WORKING_DIR}final_ranker_model_fold_{fold}.json'
#     model = xgb.XGBRanker()
#     model.load_model(model_path)
#     # Predict outputs ranking scores
#     test_predictions += model.predict(X_test) / N_SPLITS

# # Create submission file
# print("\nCreating submission file...")
# submission_df = submission_ids.copy()
# submission_df['id5'] = pd.to_datetime(submission_df['id5'], errors='coerce').dt.strftime('%m-%d-%Y')
# cleaned_predictions = np.nan_to_num(test_predictions, nan=-999) # Use a low score for NaNs
# submission_df['pred'] = cleaned_predictions

# submission_path = f'{WORKING_DIR}r2_submission_final_ranker18.csv'
# submission_df.to_csv(submission_path, index=False)
# print(f"\nSubmission file successfully saved to: {submission_path}")
# print("\nFirst 5 rows of the submission file:")
# print(submission_df.head())

--- STAGE 1: Loading and Preparing Data for XGBRanker ---
Successfully loaded final training (v92) and test (v92) data.
Preprocessing data for training...
Converting all feature columns to numeric types...

Optimizing memory usage for training data...
Memory usage of dataframe is 2367.98 MB
Memory usage after optimization is: 1217.04 MB
Decreased by 48.6%

Optimizing memory usage for test data...
Memory usage of dataframe is 1132.30 MB
Memory usage after optimization is: 600.84 MB
Decreased by 46.9%
Prepared training data with 439 features and 52468 groups.

--- STAGE 2: Training Final Ensemble of XGBRanker Models ---
Starting final training with 3-Fold GroupKFold on GPU...
--- Fold 1/3 ---
Fold 1 MAP@7 Score: 0.06282
Model for fold 1 saved to /kaggle/working/final_ranker_model_fold_1.json
--- Fold 2/3 ---
Fold 2 MAP@7 Score: 0.06610
Model for fold 2 saved to /kaggle/working/final_ranker_model_fold_2.json
--- Fold 3/3 ---
Fold 3 MAP@7 Score: 0.06171
Model for fold 3 saved to /kaggle/wo

In [ ]:
# import pandas as pd
# from sklearn.preprocessing import MinMaxScaler
# import warnings

# # Suppress warnings for cleaner output
# warnings.filterwarnings('ignore')

# # --- Define Kaggle File Paths ---
# WORKING_DIR = '/kaggle/working/'

# # =============================================================================
# # STAGE 1: LOAD THE SUBMISSION FILE
# # =============================================================================
# print("--- STAGE 1: Loading the submission file ---")
# try:
#     submission_path = f'{WORKING_DIR}r2_submission_final_ranker18.csv'
#     df_sub = pd.read_csv(submission_path)
#     print("Successfully loaded submission file.")
#     print("\nOriginal 'pred' column stats:")
#     print(df_sub['pred'].describe())
# except FileNotFoundError:
#     print(f"Error: {submission_path} not found.")
#     print("Please ensure you have run the prediction script successfully.")
#     exit()

# # =============================================================================
# # STAGE 2: NORMALIZE THE PREDICTION COLUMN
# # =============================================================================
# print("\n--- STAGE 2: Normalizing the 'pred' column ---")

# # Initialize the Min-Max Scaler to scale values between 0 and 1
# scaler = MinMaxScaler()

# # The scaler expects a 2D array, so we need to reshape the column
# # We fit and transform in one step
# df_sub['pred_normalized'] = scaler.fit_transform(df_sub[['pred']])

# print("\n'pred' column has been normalized.")
# print("\nNew 'pred_normalized' column stats:")
# print(df_sub['pred_normalized'].describe())


# # =============================================================================
# # STAGE 3: CREATE AND SAVE THE NORMALIZED SUBMISSION FILE
# # =============================================================================
# print("\n--- STAGE 3: Saving the normalized submission file ---")

# # Create the final dataframe, dropping the old prediction column
# df_final_sub = df_sub[['id1', 'id2', 'id3', 'id5']].copy()
# df_final_sub['pred'] = df_sub['pred_normalized']

# # Define the output path
# output_path = f'{WORKING_DIR}r2_submission_final_ranker_normalized18.csv'

# try:
#     df_final_sub.to_csv(output_path, index=False)
#     print(f"\nSuccessfully saved normalized submission file to: {output_path}")
#     print("\nFirst 5 rows of the new submission file:")
#     print(df_final_sub.head())
# except Exception as e:
#     print(f"\nError saving the file: {e}")

In [ ]:
# # =============================================================================
# # SECTION 0: SETUP AND IMPORTS
# # =============================================================================
# print("--- Section 0: Setting up the environment ---")
# # Install the necessary libraries for graph and sequential embeddings
# !pip install -q node2vec gensim

# import pandas as pd
# import numpy as np
# import dask.dataframe as dd
# import warnings
# import gc
# import os
# import networkx as nx
# from node2vec import Node2Vec
# from gensim.models import Word2Vec
# from sklearn.decomposition import PCA
# from sklearn.preprocessing import StandardScaler

# # Suppress warnings for cleaner output
# warnings.filterwarnings('ignore')

# # --- Define Kaggle File Paths ---
# INPUT_DIR = '/kaggle/input/amexkkgp/'
# WORKING_DIR = '/kaggle/working/'

# # =============================================================================
# # STAGE 1: LOAD DATA
# # =============================================================================
# print("\n--- Section 1: Loading Data ---")
# try:
#     # Load your latest training and test sets
#     df_train = pd.read_parquet(f'/kaggle/input/amex-v15/train_data_v15.parquet')
#     df_test = pd.read_parquet(f'/kaggle/input/amex-v15/test_data_v15.parquet')
    
#     # Use Dask for the large event log
#     event_dd = dd.read_parquet(f'{INPUT_DIR}add_event.parquet', columns=['id2', 'id3', 'id4'])
    
#     print("Successfully loaded all required data.")
# except FileNotFoundError:
#     print("Error: Required data not found. Please ensure all necessary files are in the correct directories.")
#     exit()

# # =============================================================================
# # TECHNIQUE 1: GRAPH EMBEDDINGS (NODEVEC) WITH PCA
# # =============================================================================
# print("\n--- Technique 1: Creating Graph Embedding Features ---")

# # --- Step 1a: Build the Interaction Graph on a Sample ---
# print("Step 1a: Building the customer-offer interaction graph on a sample...")
# # Use a large sample of the event data to make the process much faster
# event_dd_sample = event_dd.sample(frac=0.3, random_state=42)
# edge_list = event_dd_sample[['id2', 'id3']].compute()
# # Create a graph from the edge list
# G = nx.from_pandas_edgelist(edge_list, 'id2', 'id3')
# print(f"Graph created with {G.number_of_nodes()} nodes and {G.number_of_edges()} edges.")

# # --- Step 1b: Train the Node2Vec Model ---
# print("Step 1b: Training the Node2Vec model...")
# node2vec = Node2Vec(G, dimensions=32, walk_length=20, num_walks=10, workers=4, quiet=True)
# # min_count=5 acts as a form of pruning for robustness.
# model_n2v = node2vec.fit(window=5, min_count=5, batch_words=4)
# print("Node2Vec model trained successfully.")

# # --- Step 1c: Extract Offer Embeddings ---
# print("Step 1c: Extracting offer embeddings...")
# offer_ids = df_train['id3'].unique().tolist() + df_test['id3'].unique().tolist()
# offer_ids = list(set(offer_ids)) # Get unique offer IDs across train and test

# graph_embeddings = []
# for offer_id in offer_ids:
#     try:
#         vector = model_n2v.wv[offer_id]
#         graph_embeddings.append([offer_id] + vector.tolist())
#     except KeyError:
#         continue

# graph_embedding_features = pd.DataFrame(graph_embeddings, columns=['id3'] + [f'graph_emb_{i}' for i in range(32)])
# print("Graph embedding features created successfully.")

# # Clean up memory
# del G, edge_list, model_n2v, node2vec, event_dd_sample
# gc.collect()

# # =============================================================================
# # TECHNIQUE 2: SEQUENTIAL EMBEDDINGS (WORD2VEC) WITH PCA
# # =============================================================================
# print("\n--- Technique 2: Creating Sequential Embedding Features ---")

# # --- Step 2a: Create Customer Sessions ---
# print("Step 2a: Creating customer sessions...")
# event_dd['impression_time'] = dd.to_datetime(event_dd['id4'], errors='coerce')

# # MODIFIED: Use a more robust Dask method to create sessions by sorting first.
# print("Sorting events by customer and time...")
# event_dd = event_dd.sort_values(['id2', 'impression_time'])

# # Define a function to calculate time differences within partitions
# def calculate_time_diff(df):
#     df['time_diff'] = df.groupby('id2')['impression_time'].diff().dt.total_seconds()
#     return df

# # Apply the function to each partition using map_partitions
# meta = event_dd._meta.copy()
# meta['time_diff'] = pd.Series(dtype='float64')
# event_dd = event_dd.map_partitions(calculate_time_diff, meta=meta)

# # A new session starts if the time difference is > 30 minutes (1800 seconds)
# event_dd['new_session'] = (event_dd['time_diff'] > 1800).fillna(True)
# event_dd['session_id'] = event_dd.groupby('id2')['new_session'].cumsum()

# # Create the list of sessions (sequences of offers)
# print("Aggregating offers into sessions...")
# sessions = event_dd.groupby(['id2', 'session_id'])['id3'].apply(list, meta=('id3', 'object')).compute().tolist()
# print(f"Created {len(sessions)} customer sessions.")

# # --- Step 2b: Train the Word2Vec Model ---
# print("Step 2b: Training the Word2Vec model...")
# # Train the model on the sessions. min_count=5 acts as pruning.
# model_w2v = Word2Vec(sentences=sessions, vector_size=32, window=5, min_count=5, workers=4)
# print("Word2Vec model trained successfully.")

# # --- Step 2c: Extract Offer Embeddings ---
# print("Step 2c: Extracting sequential embeddings...")
# sequential_embeddings = []
# for offer_id in offer_ids:
#     try:
#         vector = model_w2v.wv[offer_id]
#         sequential_embeddings.append([offer_id] + vector.tolist())
#     except KeyError:
#         continue

# sequential_embedding_features = pd.DataFrame(sequential_embeddings, columns=['id3'] + [f'seq_emb_{i}' for i in range(32)])
# print("Sequential embedding features created successfully.")

# # Clean up memory
# del sessions, model_w2v, event_dd
# gc.collect()

# # =============================================================================
# # STAGE 3: APPLY PCA AND MERGE FEATURES
# # =============================================================================
# print("\n--- STAGE 3: Applying PCA and Merging Final Features ---")

# def apply_pca_to_embeddings(df_features, prefix):
#     """Applies PCA to a dataframe of embedding features."""
#     ids = df_features[['id3']]
#     embeddings = df_features.drop(columns=['id3'])
    
#     # Scale the data before PCA
#     scaler = StandardScaler()
#     embeddings_scaled = scaler.fit_transform(embeddings)
    
#     # Use PCA to capture 95% of the variance
#     pca = PCA(n_components=0.95)
#     embeddings_pca = pca.fit_transform(embeddings_scaled)
#     print(f"PCA for '{prefix}' completed. New shape: {embeddings_pca.shape}")
    
#     # Create a DataFrame from the PCA components
#     pca_df = pd.DataFrame(embeddings_pca, columns=[f'{prefix}_pca_{i}' for i in range(embeddings_pca.shape[1])])
#     return pd.concat([ids, pca_df], axis=1)

# # --- Apply PCA to both sets of embeddings ---
# graph_pca_features = apply_pca_to_embeddings(graph_embedding_features, 'graph')
# sequential_pca_features = apply_pca_to_embeddings(sequential_embedding_features, 'seq')

# def enrich_dataframe(df, graph_pca_features, sequential_pca_features):
#     """A reusable function to enrich a dataframe with new PCA features."""
#     df['id3'] = df['id3'].astype(str)
#     graph_pca_features['id3'] = graph_pca_features['id3'].astype(str)
#     sequential_pca_features['id3'] = sequential_pca_features['id3'].astype(str)
    
#     # Merge graph PCA features
#     df = pd.merge(df, graph_pca_features, on='id3', how='left')
#     # Merge sequential PCA features
#     df = pd.merge(df, sequential_pca_features, on='id3', how='left')
    
#     return df

# # --- Enrich both the training and test dataframes ---
# print("Enriching training data...")
# df_train_final = enrich_dataframe(df_train, graph_pca_features, sequential_pca_features)
# print("Enriching test data...")
# df_test_final = enrich_dataframe(df_test, graph_pca_features, sequential_pca_features)

# # --- Save the final DataFrames ---
# try:
#     train_output_path = f'{WORKING_DIR}train_data_v18.parquet'
#     test_output_path = f'{WORKING_DIR}test_data_v18.parquet'
    
#     df_train_final.to_parquet(train_output_path)
#     df_test_final.to_parquet(test_output_path)
    
#     print(f"\nSuccessfully saved final training data to: {train_output_path}")
#     print(f"Successfully saved final test data to: {test_output_path}")
# except Exception as e:
#     print(f"\nError saving the final files: {e}")

# print("\n--- ADVANCED EMBEDDING FEATURE ENGINEERING COMPLETE ---")

In [1]:
# =============================================================================
# SECTION 0: SETUP AND IMPORTS
# =============================================================================
print("--- Section 0: Setting up the environment ---")
# Install the necessary library for text embeddings
!pip install -q sentence-transformers

import pandas as pd
import numpy as np
import dask.dataframe as dd
import warnings
import gc
import os

from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

# Suppress warnings for cleaner output
warnings.filterwarnings('ignore')

# --- Define Kaggle File Paths ---
INPUT_DIR = '/kaggle/input/amexkkgp/'
WORKING_DIR = '/kaggle/working/'

# =============================================================================
# STAGE 1: LOAD DATA
# =============================================================================
print("\n--- Section 1: Loading Data ---")
try:
    # Load your latest training and holdout test sets with entity clusters
    df_train = pd.read_parquet(f'{WORKING_DIR}train_entity_clusters.parquet')
    df_holdout_test = pd.read_parquet(f'{WORKING_DIR}holdout_test_entity_clusters.parquet')
    
    # Load supplementary data for feature creation
    offer_meta_df = pd.read_parquet(f'{INPUT_DIR}offer_metadata.parquet')
    event_dd = dd.read_parquet(f'{INPUT_DIR}add_event.parquet')
    
    print("Successfully loaded all required data.")
    print(f"Train shape: {df_train.shape}, Holdout Test shape: {df_holdout_test.shape}")
except FileNotFoundError as e:
    print(f"Error: Required data not found: {e}")
    print("Please ensure the entity_clustering.py script ran successfully.")
    exit()

# =============================================================================
# TECHNIQUE 1: ADVANCED CO-CLICK FEATURES
# =============================================================================
print("\n--- Technique 1: Creating Advanced Co-Click Features ---")

# --- Step 1a: Create Customer Sessions on a Sample ---
print("Step 1a: Creating customer sessions on a sample of the data...")
# Use a sample of the event data to make the process manageable
event_dd_sample = event_dd.sample(frac=0.2, random_state=42)
event_dd_sample['impression_time'] = dd.to_datetime(event_dd_sample['id4'], errors='coerce')

# Sort by customer and time to correctly identify sessions
event_dd_sample = event_dd_sample.sort_values(['id2', 'impression_time'])

# Define a function to calculate time differences within partitions
def calculate_time_diff(df):
    df['time_diff'] = df.groupby('id2')['impression_time'].diff().dt.total_seconds()
    return df

# Apply the function using map_partitions for robust Dask execution
meta = event_dd_sample._meta.copy()
meta['time_diff'] = pd.Series(dtype='float64')
event_dd_sample = event_dd_sample.map_partitions(calculate_time_diff, meta=meta)

# A new session starts if the time difference is > 30 minutes (1800 seconds)
event_dd_sample['new_session'] = (event_dd_sample['time_diff'] > 1800).fillna(True)
event_dd_sample['session_id'] = event_dd_sample.groupby('id2')['new_session'].cumsum()
print("Customer sessions created.")


# --- Step 1b: Calculate Co-occurrence ---
print("Step 1b: Calculating offer co-occurrence within sessions...")
session_offers_dd = event_dd_sample[['session_id', 'id2', 'id3']]
merged_sessions_dd = dd.merge(session_offers_dd, session_offers_dd, on=['session_id', 'id2'])
co_occurrence_dd = merged_sessions_dd[merged_sessions_dd['id3_x'] != merged_sessions_dd['id3_y']]
co_occurrence_counts = co_occurrence_dd.groupby(['id3_x', 'id3_y']).size().compute().reset_index(name='count')
print("Co-occurrence calculated successfully.")


# --- Step 1c: Create Advanced Co-Click Features ---
print("Step 1c: Creating final co-click features...")
# Use the full event log to get the most accurate historical CTRs
event_dd['clicked'] = (~event_dd['id7'].isnull()).astype(int)
offer_ctr = event_dd.groupby('id3')['clicked'].mean().compute().to_dict()

# Get the CTR of each partner offer
co_occurrence_counts['partner_ctr'] = co_occurrence_counts['id3_y'].map(offer_ctr)
co_occurrence_counts = co_occurrence_counts.sort_values(['id3_x', 'count'], ascending=[True, False])

# Find the top 5 partners for each offer
top_5_partners = co_occurrence_counts.groupby('id3_x').head(5)

# Calculate aggregate stats for the top 5 partners' CTRs
co_click_agg = top_5_partners.groupby('id3_x')['partner_ctr'].agg(['mean', 'max', 'std']).reset_index()
co_click_features = co_click_agg.rename(columns={
    'id3_x': 'id3',
    'mean': 'top5_partner_mean_ctr',
    'max': 'top5_partner_max_ctr',
    'std': 'top5_partner_std_ctr'
})
print("Advanced co-click features created successfully.")

# Clean up memory
del event_dd, event_dd_sample, co_occurrence_counts, merged_sessions_dd, co_occurrence_dd
gc.collect()

# =============================================================================
# TECHNIQUE 2: TEXT SIMILARITY FEATURES
# =============================================================================
print("\n--- Technique 2: Creating Text Similarity Features ---")

# --- Step 2a: Prepare the Text Corpus and Generate Embeddings ---
print("Step 2a: Preparing text corpus and generating embeddings...")
offer_meta_df['id3'] = offer_meta_df['id3'].astype(str)
offer_meta_df['full_text'] = offer_meta_df['id9'].fillna('') + ' ' + offer_meta_df['f378'].fillna('')
offer_texts = offer_meta_df[['id3', 'id8', 'full_text']].copy()

model = SentenceTransformer('all-MiniLM-L6-v2')
embeddings = model.encode(offer_texts['full_text'].tolist(), show_progress_bar=True)
embedding_df = pd.DataFrame(embeddings, columns=[f'text_emb_{i}' for i in range(embeddings.shape[1])])
offer_embeddings_df = pd.concat([offer_texts[['id3', 'id8']], embedding_df], axis=1)
print("Text embeddings generated.")

# --- Step 2b: Create Industry DNA and Calculate Similarity ---
print("Step 2b: Calculating offer-industry text similarity...")
# Calculate the average embedding vector for each industry
industry_dna = offer_embeddings_df.groupby('id8').mean(numeric_only=True).reset_index()
industry_dna = industry_dna.rename(columns={col: f'industry_{col}' for col in industry_dna.columns if 'emb' in col})

# Merge the industry DNA back to each offer
offer_embeddings_df = pd.merge(offer_embeddings_df, industry_dna, on='id8', how='left')

# Calculate the cosine similarity between each offer's embedding and its industry's average embedding
emb_cols = [f'text_emb_{i}' for i in range(embeddings.shape[1])]
industry_emb_cols = [f'industry_{col}' for col in emb_cols] # Correctly renamed columns

# Fill NaNs in industry vectors for offers in industries with no text data
offer_embeddings_df[industry_emb_cols] = offer_embeddings_df[industry_emb_cols].fillna(0)

# Vectorized cosine similarity calculation for performance
offer_vecs = offer_embeddings_df[emb_cols].values
industry_vecs = offer_embeddings_df[industry_emb_cols].values
# Calculate dot product and norms
dot_product = np.sum(offer_vecs * industry_vecs, axis=1)
offer_norm = np.linalg.norm(offer_vecs, axis=1)
industry_norm = np.linalg.norm(industry_vecs, axis=1)
# Calculate cosine similarity, handling potential division by zero
cosine_sim = dot_product / (offer_norm * industry_norm + 1e-6)

offer_embeddings_df['offer_industry_text_similarity'] = cosine_sim
text_similarity_features = offer_embeddings_df[['id3', 'offer_industry_text_similarity']]
print("Text similarity features created successfully.")


# =============================================================================
# STAGE 3: MERGE NEW FEATURES AND SAVE
# =============================================================================
print("\n--- STAGE 3: Merging New Features and Saving Final Datasets ---")

def enrich_dataframe(df, co_click_features, text_similarity_features):
    """A reusable function to enrich a dataframe with new features."""
    df['id3'] = df['id3'].astype(str)
    co_click_features['id3'] = co_click_features['id3'].astype(str)
    text_similarity_features['id3'] = text_similarity_features['id3'].astype(str)
    
    # Merge co-click features
    df = pd.merge(df, co_click_features, on='id3', how='left')
    # Merge text similarity features
    df = pd.merge(df, text_similarity_features, on='id3', how='left')
    
    return df

# --- Enrich both the training and test dataframes ---
print("Enriching training data...")
df_train_final = enrich_dataframe(df_train.copy(), co_click_features, text_similarity_features)
print("Enriching holdout-test data...")
df_holdout_test_final = enrich_dataframe(df_holdout_test.copy(), co_click_features, text_similarity_features)

# --- Save the final DataFrames ---
try:
    train_output_path = f'{WORKING_DIR}train_final_features.parquet'
    holdout_test_output_path = f'{WORKING_DIR}holdout_test_final_features.parquet'
    
    df_train_final.to_parquet(train_output_path)
    df_holdout_test_final.to_parquet(holdout_test_output_path)
    
    print(f"\nSuccessfully saved final training data to: {train_output_path}")
    print(f"Successfully saved final holdout-test data to: {holdout_test_output_path}")
    print(f"Final Train shape: {df_train_final.shape}, Final Holdout Test shape: {df_holdout_test_final.shape}")
except Exception as e:
    print(f"\nError saving the final files: {e}")

print("\n--- ADVANCED FEATURE ENGINEERING COMPLETE ---")

--- Section 0: Setting up the environment ---


2025-08-31 17:36:38.053019: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1756661798.240855     179 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1756661798.293780     179 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered



--- Section 1: Loading Data ---
Successfully loaded all required data.
Train shape: (166729, 395), Holdout Test shape: (55577, 395)

--- Technique 1: Creating Advanced Co-Click Features ---
Step 1a: Creating customer sessions on a sample of the data...
Customer sessions created.
Step 1b: Calculating offer co-occurrence within sessions...
Co-occurrence calculated successfully.
Step 1c: Creating final co-click features...
Advanced co-click features created successfully.

--- Technique 2: Creating Text Similarity Features ---
Step 2a: Preparing text corpus and generating embeddings...


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/131 [00:00<?, ?it/s]

Text embeddings generated.
Step 2b: Calculating offer-industry text similarity...
Text similarity features created successfully.

--- STAGE 3: Merging New Features and Saving Final Datasets ---
Enriching training data...
Enriching holdout-test data...

Successfully saved final training data to: /kaggle/working/train_final_features.parquet
Successfully saved final holdout-test data to: /kaggle/working/holdout_test_final_features.parquet
Final Train shape: (166729, 399), Final Holdout Test shape: (55577, 399)

--- ADVANCED FEATURE ENGINEERING COMPLETE ---


In [2]:
# =============================================================================
# SECTION 0: SETUP AND IMPORTS
# =============================================================================
print("--- Section 0: Setting up the environment ---")
# Install the necessary library for text embeddings
!pip install -q sentence-transformers

import pandas as pd
import numpy as np
import warnings
import gc
import os

from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from tqdm.auto import tqdm

# Suppress warnings for cleaner output
warnings.filterwarnings('ignore')
tqdm.pandas()

# --- Define Kaggle File Paths ---
INPUT_DIR = '/kaggle/input/amexkkgp/'
WORKING_DIR = '/kaggle/working/'

# =============================================================================
# STAGE 1: LOAD DATA
# =============================================================================
print("\n--- Section 1: Loading Data ---")
try:
    # Load the datasets you just created in the previous step
    df_train = pd.read_parquet(f'{WORKING_DIR}train_final_features.parquet')
    df_holdout_test = pd.read_parquet(f'{WORKING_DIR}holdout_test_final_features.parquet')
    
    # Load supplementary data needed for embeddings
    offer_meta_df = pd.read_parquet(f'{INPUT_DIR}offer_metadata.parquet')
    
    print("Successfully loaded all required data.")
    print(f"Train shape: {df_train.shape}, Holdout Test shape: {df_holdout_test.shape}")
except FileNotFoundError as e:
    print(f"Error: Required data not found: {e}")
    print("Please ensure the final_embedding_features.py script ran successfully.")
    exit()

# =============================================================================
# STAGE 2: GENERATE TEXT EMBEDDINGS (needed for similarity calculation)
# =============================================================================
print("\n--- Stage 2: Generating Text Embeddings for Similarity Calculation ---")

offer_meta_df['id3'] = offer_meta_df['id3'].astype(str)
offer_meta_df['full_text'] = offer_meta_df['id9'].fillna('') + ' ' + offer_meta_df['f378'].fillna('')
offer_texts = offer_meta_df[['id3', 'full_text']].copy()

model = SentenceTransformer('all-MiniLM-L6-v2')
embeddings = model.encode(offer_texts['full_text'].tolist(), show_progress_bar=True)
embedding_df = pd.DataFrame(embeddings, columns=[f'text_emb_{i}' for i in range(embeddings.shape[1])])
offer_embeddings_df = pd.concat([offer_texts[['id3']], embedding_df], axis=1)
print("Text embeddings generated for all offers.")

# =============================================================================
# STAGE 3: SIMILARITY-BASED SEQUENTIAL FEATURES (RNN-like Logic)
# =============================================================================
print("\n--- Stage 3: Creating Similarity-Based Sequential Features ---")

def create_sequential_features(df, offer_embeddings_df):
    """
    Creates handcrafted features based on the similarity of a target offer
    to a customer's recent click history within the provided dataframe.
    """
    print(f"Preparing data for sequential feature engineering on a dataframe of shape {df.shape}...")
    # Ensure time is in the correct format
    df['impression_time'] = pd.to_datetime(df['id4'], errors='coerce')
    
    # Create a lookup dictionary for offer embeddings for fast access
    emb_cols = [col for col in offer_embeddings_df if 'text_emb' in col]
    offer_embeddings_lookup = offer_embeddings_df.set_index('id3')[emb_cols]

    # Get a dataframe of only the click events to build histories
    clicks = df[df['y'] == 1].copy()
    clicks = clicks.sort_values('impression_time')
    
    # For each customer, create a list of their clicked offers and timestamps
    click_history = clicks.groupby('id2').agg({
        'id3': list,
        'impression_time': list
    }).rename(columns={'id3': 'click_history_offers', 'impression_time': 'click_history_times'}).reset_index()

    # Merge this history back to the main dataframe
    df = pd.merge(df, click_history, on='id2', how='left')
    
    def calculate_similarity_features(row, N=5):
        # Get current offer details
        current_offer_id = row['id3']
        current_time = row['impression_time']
        
        # Check if history exists
        if not isinstance(row['click_history_offers'], list):
            return pd.Series([np.nan] * 4, index=['max_sim_recent_5', 'mean_sim_recent_5', 'std_sim_recent_5', 'time_weighted_mean_sim_recent_5'])
            
        # Combine history into a dataframe
        history_df = pd.DataFrame({
            'offer': row['click_history_offers'],
            'time': row['click_history_times']
        })
        
        # Filter for clicks that happened *before* the current impression
        recent_history = history_df[history_df['time'] < current_time].tail(N)
        
        if recent_history.empty:
            return pd.Series([np.nan] * 4, index=['max_sim_recent_5', 'mean_sim_recent_5', 'std_sim_recent_5', 'time_weighted_mean_sim_recent_5'])

        # Get embeddings for current offer and recent clicks
        try:
            current_vec = offer_embeddings_lookup.loc[current_offer_id].values.reshape(1, -1)
            recent_vecs = offer_embeddings_lookup.loc[recent_history['offer']].values
        except KeyError:
             return pd.Series([np.nan] * 4, index=['max_sim_recent_5', 'mean_sim_recent_5', 'std_sim_recent_5', 'time_weighted_mean_sim_recent_5'])

        # Calculate similarities and time differences
        sims = cosine_similarity(current_vec, recent_vecs)[0]
        time_diffs_days = (current_time - recent_history['time']).dt.total_seconds() / (24 * 3600)
        
        # Time decay factor: more recent clicks get higher weight (e.g., exponential decay)
        time_weights = np.exp(-time_diffs_days)

        # Calculate final features
        max_sim = np.max(sims)
        mean_sim = np.mean(sims)
        std_sim = np.std(sims) if len(sims) > 1 else 0 # Std dev is 0 if only one item
        time_weighted_mean_sim = np.sum(sims * time_weights) / np.sum(time_weights) if np.sum(time_weights) > 0 else np.nan
        
        return pd.Series([max_sim, mean_sim, std_sim, time_weighted_mean_sim], index=['max_sim_recent_5', 'mean_sim_recent_5', 'std_sim_recent_5', 'time_weighted_mean_sim_recent_5'])

    print(f"Applying similarity logic to {len(df)} rows. This may take a while...")
    # Apply the function to each row
    sequential_features = df.progress_apply(calculate_similarity_features, axis=1)
    
    # Combine the new features with the original dataframe
    df = pd.concat([df, sequential_features], axis=1)
    
    # Clean up temporary columns
    df = df.drop(columns=['click_history_offers', 'click_history_times', 'impression_time'])
    
    return df

# --- Create sequential features for both train and test data ---
df_train_final = create_sequential_features(df_train, offer_embeddings_df)
df_holdout_test_final = create_sequential_features(df_holdout_test, offer_embeddings_df)
print("Similarity-based sequential features created successfully.")

# =============================================================================
# STAGE 4: SAVE THE FINAL, FULLY-FEATURED DATASETS
# =============================================================================
print("\n--- STAGE 4: Saving Final, Fully-Featured Datasets ---")

try:
    train_output_path = f'{WORKING_DIR}train_fully_featured.parquet'
    holdout_test_output_path = f'{WORKING_DIR}holdout_test_fully_featured.parquet'
    
    df_train_final.to_parquet(train_output_path)
    df_holdout_test_final.to_parquet(holdout_test_output_path)
    
    print(f"\nSuccessfully saved final training data to: {train_output_path}")
    print(f"Successfully saved final holdout-test data to: {holdout_test_output_path}")
    print(f"Final Train shape: {df_train_final.shape}, Final Holdout Test shape: {df_holdout_test_final.shape}")
except Exception as e:
    print(f"\nError saving the final files: {e}")

print("\n--- FULL FEATURE ENGINEERING PIPELINE COMPLETE ---")

--- Section 0: Setting up the environment ---


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



--- Section 1: Loading Data ---
Successfully loaded all required data.
Train shape: (166729, 399), Holdout Test shape: (55577, 399)

--- Stage 2: Generating Text Embeddings for Similarity Calculation ---


Batches:   0%|          | 0/131 [00:00<?, ?it/s]

Text embeddings generated for all offers.

--- Stage 3: Creating Similarity-Based Sequential Features ---
Preparing data for sequential feature engineering on a dataframe of shape (166729, 399)...
Applying similarity logic to 166729 rows. This may take a while...


  0%|          | 0/166729 [00:00<?, ?it/s]

Preparing data for sequential feature engineering on a dataframe of shape (55577, 399)...
Applying similarity logic to 55577 rows. This may take a while...


  0%|          | 0/55577 [00:00<?, ?it/s]

Similarity-based sequential features created successfully.

--- STAGE 4: Saving Final, Fully-Featured Datasets ---

Successfully saved final training data to: /kaggle/working/train_fully_featured.parquet
Successfully saved final holdout-test data to: /kaggle/working/holdout_test_fully_featured.parquet
Final Train shape: (166729, 403), Final Holdout Test shape: (55577, 403)

--- FULL FEATURE ENGINEERING PIPELINE COMPLETE ---


In [4]:
# =============================================================================
# SECTION 0: SETUP AND IMPORTS
# =============================================================================
print("--- Section 0: Setting up the environment ---")
# Install the necessary library for CatBoost
!pip install -q catboost

import pandas as pd
import numpy as np
from catboost import CatBoostRanker, Pool
from sklearn.model_selection import GroupKFold
import warnings
import gc
import os

# Suppress warnings for cleaner output
warnings.filterwarnings('ignore')

# --- Define Kaggle File Paths ---
INPUT_DIR = '/kaggle/input/amexkkgp/'
# Assumes the output of the previous script is in the working directory
WORKING_DIR = '/kaggle/working/'

# =============================================================================
# STAGE 0: DEFINE THE MAP@7 EVALUATION METRIC
# =============================================================================
def map_at_k(y_true, y_pred, group_ids, k=7):
    """
    Calculates the Mean Average Precision at k.
    """
    df = pd.DataFrame({'group': group_ids, 'y_true': y_true, 'y_pred': y_pred})
    
    average_precisions = []
    for group in df['group'].unique():
        group_df = df[df['group'] == group].sort_values('y_pred', ascending=False).reset_index(drop=True)
        
        # Get the top k predictions
        group_df = group_df.head(k)
        
        if group_df['y_true'].sum() == 0:
            average_precisions.append(0)
            continue
            
        relevant_hits = 0
        precision_at_i = []
        for i, row in group_df.iterrows():
            if row['y_true'] == 1:
                relevant_hits += 1
                precision_at_i.append(relevant_hits / (i + 1))
        
        if not precision_at_i:
            average_precisions.append(0)
        else:
            average_precisions.append(np.mean(precision_at_i))
            
    return np.mean(average_precisions)


# =============================================================================
# STAGE 1: DATA LOADING AND PREPARATION FOR CATBOOSTRANKER
# =============================================================================
print("--- STAGE 1: Loading and Preparing Data for CatBoostRanker ---")
try:
    # Load your final, pre-cleaned training and test data from the previous step
    df_train = pd.read_parquet(f'{WORKING_DIR}train_fully_featured.parquet')
    df_holdout_test = pd.read_parquet(f'{WORKING_DIR}holdout_test_fully_featured.parquet')
    print("Successfully loaded final training and holdout-test data.")
except FileNotFoundError:
    print("Error: Fully featured data not found. Please run the final feature engineering script first.")
    exit()

# --- Memory Reduction Function ---
def reduce_mem_usage(df):
    """Iterate through all the columns of a dataframe and modify the data type to reduce memory usage."""
    start_mem = df.memory_usage().sum() / 1024**2
    print(f'Memory usage of dataframe is {start_mem:.2f} MB')
    
    for col in df.columns:
        col_type = df[col].dtype
        if str(col_type)[:3] == 'int' or str(col_type)[:5] == 'float':
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
    
    end_mem = df.memory_usage().sum() / 1024**2
    print(f'Memory usage after optimization is: {end_mem:.2f} MB')
    print(f'Decreased by {100 * (start_mem - end_mem) / start_mem:.1f}%')
    return df

# --- Prepare Feature and Target Names ---
target_col = 'y'
# All columns are features except for identifiers and the target
feature_cols = [col for col in df_train.columns if col not in ['id1', 'id2', 'id3', 'id4', 'id5', 'y', 'id8']]
# Identify all categorical features for CatBoost's internal handling
categorical_features = [col for col in df_train.columns if 'cluster' in col or col == 'offer_type_code' or col == 'offer_type_x_weekend']

# --- Preprocessing ---
print("Preprocessing data for training...")
# Convert feature columns to numeric, let CatBoost handle categoricals as objects
for col in feature_cols:
    if col not in categorical_features:
        df_train[col] = pd.to_numeric(df_train[col], errors='coerce')
        df_holdout_test[col] = pd.to_numeric(df_holdout_test[col], errors='coerce')

# Convert categorical columns to string type for CatBoost
for col in categorical_features:
    df_train[col] = df_train[col].astype(str).fillna('missing')
    df_holdout_test[col] = df_holdout_test[col].astype(str).fillna('missing')

# Final imputation for numeric features
numeric_features = [col for col in feature_cols if col not in categorical_features]
df_train[numeric_features] = df_train[numeric_features].fillna(-999)
df_holdout_test[numeric_features] = df_holdout_test[numeric_features].fillna(-999)


# Optimize memory usage
print("\nOptimizing memory usage for training data...")
df_train = reduce_mem_usage(df_train)
print("\nOptimizing memory usage for holdout-test data...")
df_holdout_test = reduce_mem_usage(df_holdout_test)

# --- Create Groups for CatBoostRanker ---
# A "query" or "group" is a single ranking task (all offers for one customer on one day)
df_train['group_id'] = df_train['id2'].astype(str) + '_' + df_train['id5'].astype(str)
df_train = df_train.sort_values('group_id').reset_index(drop=True)

# Define features (X) and target (y)
X_train = df_train[feature_cols]
y_train = df_train[target_col].astype(int)
print(f"Prepared training data with {len(feature_cols)} features and {df_train['group_id'].nunique()} groups.")

# Prepare holdout-test set
df_holdout_test['group_id'] = df_holdout_test['id2'].astype(str) + '_' + df_holdout_test['id5'].astype(str)
X_holdout_test = df_holdout_test[feature_cols]
y_holdout_test = df_holdout_test[target_col].astype(int)

# =============================================================================
# STAGE 2: TRAINING FINAL ENSEMBLE WITH CATBOOSTRANKER
# =============================================================================
print("\n--- STAGE 2: Training Final Ensemble of CatBoostRanker Models ---")

# Define CatBoostRanker parameters for a balance of speed and performance
final_params = {
    'objective': 'YetiRank',
    'eval_metric': 'MAP:top=7', # CORRECTED: Changed 'k=7' to 'top=7' for correct CatBoost syntax
    'random_seed': 42,
    'task_type': 'GPU',
    'iterations': 1500, # A reasonable number with early stopping
    'learning_rate': 0.05,
    'depth': 7, # A slightly shallower tree for speed
    'l2_leaf_reg': 3,
    'verbose': 200
}

N_SPLITS = 3 # Using 3 folds for a good balance of stability and speed
gkf_train = GroupKFold(n_splits=N_SPLITS)
oof_predictions = np.zeros(len(df_train))
oof_map_scores = []

print(f"Starting final training with {N_SPLITS}-Fold GroupKFold on GPU...")
# For CatBoostRanker, we must use the customer ID for the GroupKFold split
groups_for_split = df_train['id2']

for fold, (train_idx, val_idx) in enumerate(gkf_train.split(X_train, y_train, groups=groups_for_split)):
    print(f"--- Fold {fold+1}/{N_SPLITS} ---")
    
    X_train_fold, y_train_fold = X_train.iloc[train_idx], y_train.iloc[train_idx]
    X_val_fold, y_val_fold = X_train.iloc[val_idx], y_train.iloc[val_idx]
    
    # Get the group_id for each set
    train_group_ids = df_train.iloc[train_idx]['group_id']
    val_group_ids = df_train.iloc[val_idx]['group_id']
    
    # Create CatBoost Pool objects for efficiency
    train_pool = Pool(data=X_train_fold, label=y_train_fold, group_id=train_group_ids, cat_features=categorical_features)
    val_pool = Pool(data=X_val_fold, label=y_val_fold, group_id=val_group_ids, cat_features=categorical_features)
    
    model = CatBoostRanker(**final_params)
    model.fit(train_pool,
              eval_set=val_pool,
              early_stopping_rounds=50)
    
    # CatBoostRanker outputs scores, not probabilities
    val_preds = model.predict(X_val_fold)
    oof_predictions[val_idx] = val_preds

    # --- Calculate and store MAP@7 for this fold ---
    fold_map_score = map_at_k(y_val_fold, val_preds, val_group_ids, k=7)
    oof_map_scores.append(fold_map_score)
    print(f"Fold {fold+1} MAP@7 Score: {fold_map_score:.5f}")

    model_path = f'{WORKING_DIR}final_catboost_ranker_fold_{fold+1}.cbm'
    model.save_model(model_path)
    print(f"Model for fold {fold+1} saved to {model_path}")
    del X_train_fold, y_train_fold, X_val_fold, y_val_fold, model, train_pool, val_pool
    gc.collect()

# --- Evaluate Overall Performance ---
print(f"\n--- Overall Cross-Validation Results ---")
print(f"Mean Out-of-Fold (OOF) MAP@7 Score: {np.mean(oof_map_scores):.5f}")
print("This score is a reliable estimate of your model's generalization ability.")


# =============================================================================
# STAGE 3: FINAL EVALUATION ON HOLDOUT-TEST SET
# =============================================================================
print("\n--- STAGE 3: Final Evaluation on Holdout-Test Set ---")

# Load models and predict on the holdout test set
holdout_test_predictions = np.zeros(len(X_holdout_test))
for fold in range(1, N_SPLITS + 1):
    print(f"Predicting with Fold {fold}/{N_SPLITS}...")
    model_path = f'{WORKING_DIR}final_catboost_ranker_fold_{fold}.cbm'
    model = CatBoostRanker()
    model.load_model(model_path)
    # Predict outputs ranking scores
    holdout_test_predictions += model.predict(X_holdout_test) / N_SPLITS

# --- Calculate final MAP@7 on the holdout set ---
holdout_group_ids = df_holdout_test['group_id']
final_holdout_map_score = map_at_k(y_holdout_test, holdout_test_predictions, holdout_group_ids, k=7)
print(f"\n--- FINAL MODEL PERFORMANCE ---")
print(f"Final MAP@7 Score on Hold-Out Test Set: {final_holdout_map_score:.5f}")

# =============================================================================
# STAGE 4: SAVE HOLDOUT PREDICTIONS FOR INSPECTION
# =============================================================================
print("\n--- STAGE 4: Saving Holdout Predictions for Inspection ---")

# Create submission-style dataframe for the holdout set
holdout_preds_df = df_holdout_test[['id1', 'id2', 'id3', 'id5', 'y']].copy()
holdout_preds_df['id5'] = pd.to_datetime(holdout_preds_df['id5'], errors='coerce').dt.strftime('%m/%d/%Y')
cleaned_predictions = np.nan_to_num(holdout_test_predictions, nan=-999)
holdout_preds_df['pred_score'] = cleaned_predictions

output_path = f'{WORKING_DIR}holdout_predictions.csv'
holdout_preds_df.to_csv(output_path, index=False)
print(f"\nHoldout predictions successfully saved to: {output_path}")
print("\nFirst 5 rows of the predictions file:")
print(holdout_preds_df.head())

--- Section 0: Setting up the environment ---


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


--- STAGE 1: Loading and Preparing Data for CatBoostRanker ---
Successfully loaded final training and holdout-test data.
Preprocessing data for training...

Optimizing memory usage for training data...
Memory usage of dataframe is 512.00 MB
Memory usage after optimization is: 263.95 MB
Decreased by 48.4%

Optimizing memory usage for holdout-test data...
Memory usage of dataframe is 170.67 MB
Memory usage after optimization is: 87.98 MB
Decreased by 48.4%
Prepared training data with 396 features and 27340 groups.

--- STAGE 2: Training Final Ensemble of CatBoostRanker Models ---
Starting final training with 3-Fold GroupKFold on GPU...
--- Fold 1/3 ---
Groupwise loss function. OneHotMaxSize set to 10


Default metric period is 5 because PFound, MAP is/are not implemented for GPU
Metric PFound is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time
Metric MAP:top=7 is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time


0:	learn: 0.1209213	test: 0.1211307	best: 0.1211307 (0)	total: 14.9s	remaining: 6h 13m 13s
200:	learn: 0.1329339	test: 0.1309675	best: 0.1309675 (200)	total: 18.3s	remaining: 1m 58s
400:	learn: 0.1346908	test: 0.1320096	best: 0.1320096 (400)	total: 21.7s	remaining: 59.3s
bestTest = 0.1323617169
bestIteration = 495
Shrink model to first 496 iterations.
Fold 1 MAP@7 Score: 0.14210
Model for fold 1 saved to /kaggle/working/final_catboost_ranker_fold_1.cbm
--- Fold 2/3 ---
Groupwise loss function. OneHotMaxSize set to 10
0:	learn: 0.1176238	test: 0.1191104	best: 0.1191104 (0)	total: 30.2ms	remaining: 45.3s


Default metric period is 5 because PFound, MAP is/are not implemented for GPU
Metric PFound is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time
Metric MAP:top=7 is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time


200:	learn: 0.1317187	test: 0.1325895	best: 0.1325895 (200)	total: 3.38s	remaining: 21.9s
bestTest = 0.1331174893
bestIteration = 254
Shrink model to first 255 iterations.
Fold 2 MAP@7 Score: 0.14426
Model for fold 2 saved to /kaggle/working/final_catboost_ranker_fold_2.cbm
--- Fold 3/3 ---
Groupwise loss function. OneHotMaxSize set to 10
0:	learn: 0.1183993	test: 0.1141419	best: 0.1141419 (0)	total: 30.1ms	remaining: 45.1s


Default metric period is 5 because PFound, MAP is/are not implemented for GPU
Metric PFound is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time
Metric MAP:top=7 is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time


200:	learn: 0.1336136	test: 0.1301424	best: 0.1301965 (173)	total: 3.43s	remaining: 22.2s
400:	learn: 0.1353007	test: 0.1316308	best: 0.1316308 (400)	total: 6.8s	remaining: 18.6s
bestTest = 0.132031707
bestIteration = 527
Shrink model to first 528 iterations.
Fold 3 MAP@7 Score: 0.14160
Model for fold 3 saved to /kaggle/working/final_catboost_ranker_fold_3.cbm

--- Overall Cross-Validation Results ---
Mean Out-of-Fold (OOF) MAP@7 Score: 0.14265
This score is a reliable estimate of your model's generalization ability.

--- STAGE 3: Final Evaluation on Holdout-Test Set ---
Predicting with Fold 1/3...
Predicting with Fold 2/3...
Predicting with Fold 3/3...

--- FINAL MODEL PERFORMANCE ---
Final MAP@7 Score on Hold-Out Test Set: 0.15159

--- STAGE 4: Saving Holdout Predictions for Inspection ---

Holdout predictions successfully saved to: /kaggle/working/holdout_predictions.csv

First 5 rows of the predictions file:
                                               id1      id2      id3  \
0 